In [1]:
#librerías
import geopandas as gpd
import pandas as pd
import numpy as np
import sys
sys.path.append(r"C:\Users\mfpen\Documents\Repositorios\geoloc2\preparacion_datos\src")
from preparacion_inter_puntos import prep
sys.path.append(r"C:\Users\mfpen\Documents\Repositorios\geoloc2\preparacion_datos")
from funciones_cruces_limpieza import *

In [2]:
#Rutas de bases
curt_path = r"H:\.shortcut-targets-by-id\1xy7wVmj8pxJHqeyV9CG1vOBQs3emChdq\Geo\Data\bases_shp_INEGI\CURT\024 Atlacomulco\ATLACOMULCO_PREDIOS.shp"
manzanas_path=r"H:\.shortcut-targets-by-id\1xy7wVmj8pxJHqeyV9CG1vOBQs3emChdq\Geo\Data\bases_shp_INEGI\INFORMACION PARA INTELIGENCIA FISCAL\ATLACOMULCO\SHAPE\MANZANAS\024 Atlacomulco\2023 024 MZ.shp"
predial1_path = r"D:\Secretaria\cruces_bases\Atlacomulco\predial_diciembre_atlacomulco.xlsx"
predial_path = r"D:\Secretaria\cruces_bases\Atlacomulco\Atlacomulco_PREDIAL.xlsx"
deudores_path = r"D:\Secretaria\cruces_bases\Atlacomulco\Atlacomulco_PREDIALADEUDO.xlsx"
propietarios_path=r"D:\Secretaria\cruces_bases\Atlacomulco\Atlacomulco_PROPIETARIO.xlsx"

Lectura de las bases

In [3]:
#base de curt y manzanas municipales
CURT_mun = gpd.read_file(curt_path)
manzanas_mun = gpd.read_file(manzanas_path)
#Base predial de OneDrive
base_predial_OD = pd.read_excel(predial1_path)
#Bases de predial
pred_pred = pd.read_excel(predial_path)
pred_prop = pd.read_excel(propietarios_path)
pred_deu = pd.read_excel(deudores_path)

Cruce 1: Base CURT x Base predial de OneDrive

In [4]:
#Homologar nombres de Clave Catastral x base para los cruces
CURT_mun.rename(columns={'id_cat':'CLAVECATASTRAL'},inplace=True)
manzanas_mun.rename(columns={'cve_cat':'CLAVECATASTRAL'},inplace=True)

In [5]:
#revisar si hay vacíos en clave catastral
CURT_mun[CURT_mun.CLAVECATASTRAL.isna()]

,cve_ent,cve_mun,cve_loc,CLAVECATASTRAL,curt,notas,geometry


In [6]:
base_predial_OD.shape

(23141, 19)

In [7]:
base_predial_OD=base_predial_OD.reset_index(drop=True)
base_predial_OD.shape

(23141, 19)

In [8]:
base_predial_OD.columns

Index(['fol_rec', 'nombre', 'rfc', 'cve_catastral', 'ubicacion',
       'domicilio_fiscal', 'sup_terreno', 'sup_construc', 'valor_catastral',
       'periodo', 'periodo_anual', 'fec_pago', 'importe', 'recargos', 'multas',
       'gast_ejec', 'descuento', 'fol_fisc_uuid', 'facturado'],
      dtype='object')

In [9]:
#Se puede utilizar esta celda en caso de que la CC no este homologada a CLAVECATASTRAL
base_predial_OD.rename(columns={'cve_catastral': 'CLAVECATASTRAL'}, inplace=True)
base_predial_OD.shape

(23141, 19)

In [10]:
#Se crea una base con los registros que SI tienen CC en predial
base_predial=base_predial_OD[base_predial_OD.CLAVECATASTRAL.notna()]
base_predial.shape

(23044, 19)

In [11]:
#quitar duplicados
base_predial.drop_duplicates(subset='CLAVECATASTRAL', inplace=True)
base_predial.shape

(15766, 19)

In [12]:
#Se crean la CC a nivel manzana y a nivel predio de la base predial 
base_predial['CLAVE_MANZ']=base_predial.CLAVECATASTRAL.astype(str).str[0:8]+"00"
base_predial['CLAVE_PREDIO']=base_predial.CLAVECATASTRAL.astype(str).str[0:10]
base_predial['CLAVE_CATASTRAL']=base_predial.CLAVECATASTRAL.astype(str).str[0:10]
base_predial.tail(3)

,fol_rec,nombre,rfc,CLAVECATASTRAL,ubicacion,domicilio_fiscal,sup_terreno,sup_construc,valor_catastral,periodo,periodo_anual,fec_pago,importe,recargos,multas,gast_ejec,descuento,fol_fisc_uuid,facturado,CLAVE_MANZ,CLAVE_PREDIO,CLAVE_CATASTRAL
23135,TM049723,DIAZ RAMIREZ JESUS*,XAXX010101000,0240119813000000,LEGISLATURA,NaN,130.0,186.0,744601,DE 01-2022 AL 12-2022,2022,2022-12-30,1086.0,120.0,0,0,120.0,NaN,NaN,0240119800,0240119813,0240119813
23136,TM049724,COLIN FLORES ELOISA*,XAXX010101000,0240107919000000,"ISIDRO FABELA SUR # 27 - S/N, COL. CENTRO, C.P...",NaN,250.0,79.0,382309,DE 01-2019 AL 12-2019,2019,2022-12-30,504.0,252.0,0,0,252.0,NaN,NaN,0240107900,0240107919,0240107919
23140,TM049726,COLIN FLORES ELOISA*,XAXX010101000,0240119818000000,LEGISLATURA S/N,NaN,152.0,0.0,115520,DE 01-2022 AL 12-2022,2022,2022-12-30,208.0,35.0,0,0,35.0,NaN,NaN,0240119800,0240119818,0240119818


In [13]:
base_predial.shape

(15766, 22)

In [14]:
#Se usa esta función por si hay alguna columna con Nan o 0 o vacías y no llenarnos de columnas
base_predial=delete_empty(base_predial)
base_predial.shape

(15766, 21)

In [15]:
len(base_predial.CLAVECATASTRAL.unique())

15766

In [16]:
base_predial.columns[base_predial.columns.isin(CURT_mun.columns)]
falta= base_predial.columns[~base_predial.columns.isin(CURT_mun.columns)]
falta

Index(['fol_rec', 'nombre', 'rfc', 'ubicacion', 'domicilio_fiscal',
       'sup_terreno', 'sup_construc', 'valor_catastral', 'periodo',
       'periodo_anual', 'fec_pago', 'importe', 'recargos', 'multas',
       'descuento', 'fol_fisc_uuid', 'facturado', 'CLAVE_MANZ', 'CLAVE_PREDIO',
       'CLAVE_CATASTRAL'],
      dtype='object')

In [17]:
cruce_predial = pd.merge(base_predial[['fol_rec', 'nombre', 'rfc', 'ubicacion', 'domicilio_fiscal',
                                   'sup_terreno', 'sup_construc', 'valor_catastral', 'periodo',
                                   'periodo_anual', 'fec_pago', 'importe', 'recargos', 'multas',
                                   'descuento', 'fol_fisc_uuid', 'facturado', 'CLAVE_MANZ', 'CLAVE_PREDIO',
                                   'CLAVE_CATASTRAL','CLAVECATASTRAL']],CURT_mun, on="CLAVECATASTRAL", how='inner')
cruce_predial.shape

(1195, 27)

In [18]:
#Revisar valores unicos x CC
len(cruce_predial.CLAVECATASTRAL.unique())

1173

In [19]:
cruce_predial.columns

Index(['fol_rec', 'nombre', 'rfc', 'ubicacion', 'domicilio_fiscal',
       'sup_terreno', 'sup_construc', 'valor_catastral', 'periodo',
       'periodo_anual', 'fec_pago', 'importe', 'recargos', 'multas',
       'descuento', 'fol_fisc_uuid', 'facturado', 'CLAVE_MANZ', 'CLAVE_PREDIO',
       'CLAVE_CATASTRAL', 'CLAVECATASTRAL', 'cve_ent', 'cve_mun', 'cve_loc',
       'curt', 'notas', 'geometry'],
      dtype='object')

In [20]:
#Se crea base de todos los que no cruzaron x CURT de base predial
base_pred_n = base_predial.loc[~base_predial['CLAVECATASTRAL'].isin(cruce_predial['CLAVECATASTRAL'])]
base_pred_n.shape

(14593, 21)

In [21]:
cruce_predial.shape, base_pred_n.shape

((1195, 27), (14593, 21))

Cruce 2: Cruce de base predial a nivel predio x base CURT

In [22]:
#Se utiliza esta función para separar CC en CC manzana y CC predio
separar_digitos(CURT_mun)
CURT_mun

,cve_ent,cve_mun,cve_loc,CLAVECATASTRAL,curt,notas,geometry,CLAVE_CATASTRAL
0,15,024,0000,0240144801000000,194744919109952299131,NaN,"POLYGON ((-99.87493 19.79576, -99.87494 19.795...",0240144801
1,15,024,0000,0240109128000000,194737177109953100419,NaN,"POLYGON ((-99.88603 19.79363, -99.88612 19.793...",0240109128
2,15,024,0000,0240109116000000,194736032409953102406,NaN,"POLYGON ((-99.88603 19.79363, -99.88590 19.793...",0240109116
3,15,024,0000,0240142005000000,194824758809952102202,NaN,"POLYGON ((-99.86944 19.80691, -99.86948 19.806...",0240142005
4,15,024,0000,0240144837000000,194744232409952299340,NaN,"POLYGON ((-99.87493 19.79576, -99.87485 19.795...",0240144837
...,...,...,...,...,...,...,...,...
1919,15,024,0000,0240122320000000,194719536809952058052,NaN,"POLYGON ((-99.86814 19.78877, -99.86809 19.788...",0240122320
1920,15,024,0000,0240122321000000,194720252909952063672,NaN,"POLYGON ((-99.86820 19.78889, -99.86814 19.788...",0240122321
1921,15,024,0000,0240122328000000,194720252909952058206,NaN,"POLYGON ((-99.86827 19.78906, -99.86820 19.788...",0240122328
1922,15,024,0000,0240122345000000,194720239609952072994,NaN,"POLYGON ((-99.86859 19.78904, -99.86872 19.788...",0240122345


In [23]:
#Disolución de las geometrías a partir de clavecatastral 
CURT_mun=CURT_mun.dissolve(by='CLAVE_CATASTRAL').reset_index()
CURT_mun.tail(2)

,CLAVE_CATASTRAL,geometry,cve_ent,cve_mun,cve_loc,CLAVECATASTRAL,curt,notas
1884,0241530822,"POLYGON ((-99.84697 19.80856, -99.84700 19.808...",15,024,0000,0241530822000000,194830782609950493744,None
1885,0241530824,"POLYGON ((-99.84688 19.80852, -99.84698 19.808...",15,024,0000,0241530824000000,194830235009950491335,None


In [24]:
#Revisar que no haya vacíos
CURT_mun[CURT_mun.CLAVECATASTRAL.isna()]

,CLAVE_CATASTRAL,geometry,cve_ent,cve_mun,cve_loc,CLAVECATASTRAL,curt,notas


In [25]:
base_pred_n.columns[base_pred_n.columns.isin(CURT_mun.columns)]
falta= base_pred_n.columns[~base_pred_n.columns.isin(CURT_mun.columns)]
falta

Index(['fol_rec', 'nombre', 'rfc', 'ubicacion', 'domicilio_fiscal',
       'sup_terreno', 'sup_construc', 'valor_catastral', 'periodo',
       'periodo_anual', 'fec_pago', 'importe', 'recargos', 'multas',
       'descuento', 'fol_fisc_uuid', 'facturado', 'CLAVE_MANZ',
       'CLAVE_PREDIO'],
      dtype='object')

In [26]:
cruce_cat=pd.merge(base_pred_n[['fol_rec', 'nombre', 'rfc', 'ubicacion', 'domicilio_fiscal',
       'sup_terreno', 'sup_construc', 'valor_catastral', 'periodo',
       'periodo_anual', 'fec_pago', 'importe', 'recargos', 'multas',
       'descuento', 'fol_fisc_uuid', 'facturado', 'CLAVE_MANZ', 'CLAVE_PREDIO',
       'CLAVE_CATASTRAL','CLAVECATASTRAL']],CURT_mun, left_on="CLAVE_PREDIO", right_on="CLAVE_CATASTRAL", how='inner')
cruce_cat.shape

(520, 29)

In [27]:
#Revisión de columnas por si hay duplicadas (_x y _y)
cruce_cat.columns

Index(['fol_rec', 'nombre', 'rfc', 'ubicacion', 'domicilio_fiscal',
       'sup_terreno', 'sup_construc', 'valor_catastral', 'periodo',
       'periodo_anual', 'fec_pago', 'importe', 'recargos', 'multas',
       'descuento', 'fol_fisc_uuid', 'facturado', 'CLAVE_MANZ', 'CLAVE_PREDIO',
       'CLAVE_CATASTRAL_x', 'CLAVECATASTRAL_x', 'CLAVE_CATASTRAL_y',
       'geometry', 'cve_ent', 'cve_mun', 'cve_loc', 'CLAVECATASTRAL_y', 'curt',
       'notas'],
      dtype='object')

In [28]:
#Revisar si coinciden las claves cortadas con la completa y así decidir a cuál darle prioridad
cruce_cat[['CLAVE_CATASTRAL_x','CLAVECATASTRAL_x','CLAVE_CATASTRAL_y','CLAVECATASTRAL_y']]

,CLAVE_CATASTRAL_x,CLAVECATASTRAL_x,CLAVE_CATASTRAL_y,CLAVECATASTRAL_y
0,0240127101,02401271010F0014,0240127101,0240127101000000
1,0240127101,02401271010F0012,0240127101,0240127101000000
2,0240127101,02401271010F0002,0240127101,0240127101000000
3,0240127101,02401271010F0018,0240127101,0240127101000000
4,0240127101,02401271010F0030,0240127101,0240127101000000
...,...,...,...,...
515,0240222602,0240222602030304,0240222602,0240222602000000
516,0240222602,0240222602030102,0240222602,0240222602000000
517,0240222602,0240222602020104,0240222602,0240222602000000
518,0240222602,0240222602010301,0240222602,0240222602000000


In [29]:
cruce_cat['CLAVE_CATASTRAL']=cruce_cat.CLAVE_CATASTRAL_x.combine_first(cruce_cat.CLAVE_CATASTRAL_y)
# cruce_cat[cruce_cat.CLAVECATASTRAL.isna()]
#Eliminar columna de CLAVECATASTRAL_y y renombrar columna de CLAVECATASTRAL_x
cruce_cat.drop(['CLAVE_CATASTRAL_y','CLAVE_CATASTRAL_x'],axis=1,inplace=True )
#cruce_cat.rename(columns={'CLAVECATASTRAL_x':'CLAVECATASTRAL'},inplace=True)
cruce_cat.columns

Index(['fol_rec', 'nombre', 'rfc', 'ubicacion', 'domicilio_fiscal',
       'sup_terreno', 'sup_construc', 'valor_catastral', 'periodo',
       'periodo_anual', 'fec_pago', 'importe', 'recargos', 'multas',
       'descuento', 'fol_fisc_uuid', 'facturado', 'CLAVE_MANZ', 'CLAVE_PREDIO',
       'CLAVECATASTRAL_x', 'geometry', 'cve_ent', 'cve_mun', 'cve_loc',
       'CLAVECATASTRAL_y', 'curt', 'notas', 'CLAVE_CATASTRAL'],
      dtype='object')

In [30]:
cruce_cat['CLAVECATASTRAL']=cruce_cat.CLAVECATASTRAL_x.combine_first(cruce_cat.CLAVECATASTRAL_y)
# cruce_cat[cruce_cat.CLAVECATASTRAL.isna()]
#Eliminar columna de CLAVECATASTRAL_y y renombrar columna de CLAVECATASTRAL_x
cruce_cat.drop(['CLAVECATASTRAL_y','CLAVECATASTRAL_x'],axis=1,inplace=True )
#cruce_cat.rename(columns={'CLAVECATASTRAL_x':'CLAVECATASTRAL'},inplace=True)
cruce_cat.columns

Index(['fol_rec', 'nombre', 'rfc', 'ubicacion', 'domicilio_fiscal',
       'sup_terreno', 'sup_construc', 'valor_catastral', 'periodo',
       'periodo_anual', 'fec_pago', 'importe', 'recargos', 'multas',
       'descuento', 'fol_fisc_uuid', 'facturado', 'CLAVE_MANZ', 'CLAVE_PREDIO',
       'geometry', 'cve_ent', 'cve_mun', 'cve_loc', 'curt', 'notas',
       'CLAVE_CATASTRAL', 'CLAVECATASTRAL'],
      dtype='object')

In [31]:
len(cruce_cat.CLAVECATASTRAL.unique())

520

In [32]:
#Se vuelve a revisar que coincidan las claves a nivel predio
cruce_cat[['CLAVE_CATASTRAL','CLAVECATASTRAL']]

,CLAVE_CATASTRAL,CLAVECATASTRAL
0,0240127101,02401271010F0014
1,0240127101,02401271010F0012
2,0240127101,02401271010F0002
3,0240127101,02401271010F0018
4,0240127101,02401271010F0030
...,...,...
515,0240222602,0240222602030304
516,0240222602,0240222602030102
517,0240222602,0240222602020104
518,0240222602,0240222602010301


Cruce 3 : Cruce de base predial a nivel manzana vs (CURT x Manzanas)

In [33]:
#Se crea base de todos los que no cruzaron x CURT de base predial
base_predial_n = base_pred_n.loc[~base_pred_n['CLAVE_PREDIO'].isin(cruce_cat['CLAVE_PREDIO'])]
base_predial_n.shape

(14073, 21)

In [34]:
#Revisar que no haya vacíos
manzanas_mun[manzanas_mun.CLAVECATASTRAL.isna()]

,mun,zona,manz,CLAVECATASTRAL,geometry


In [35]:
#Se revisa el CRS de cada base para poder hacer el cruce entre ellas
print("crs CURT: ",CURT_mun.crs)
print("crs Manzanas: ",manzanas_mun.crs)

crs CURT:  EPSG:4326
crs Manzanas:  EPSG:32614


In [36]:
#Se le asigna la misma geometría de CURT a la base de manzanas
manzanas_mun=manzanas_mun.to_crs(4326)
# Se hace un overlay para la unión de las bases 
CURT_Man=CURT_mun.overlay(manzanas_mun, how='union',keep_geom_type=False)
CURT_Man.tail(5)

,CLAVE_CATASTRAL,cve_ent,cve_mun,cve_loc,CLAVECATASTRAL_1,curt,notas,mun,zona,manz,CLAVECATASTRAL_2,geometry
4307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,024,03,239,0240323900000000,"POLYGON ((-99.92435 19.81447, -99.92433 19.814..."
4308,NaN,NaN,NaN,NaN,NaN,NaN,NaN,024,01,410,0240141000000000,"POLYGON ((-99.86629 19.79529, -99.86647 19.795..."
4309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,024,01,340,0240134000000000,"POLYGON ((-99.86166 19.79180, -99.86237 19.792..."
4310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,024,01,352,0240135200000000,"MULTIPOLYGON (((-99.86557 19.78974, -99.86539 ..."
4311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,024,02,127,0240212700000000,"POLYGON ((-99.85801 19.78996, -99.85697 19.789..."


In [37]:
CURT_Man['CLAVECATASTRAL_1']=CURT_Man.CLAVECATASTRAL_1.combine_first(CURT_Man.CLAVECATASTRAL_2)
CURT_Man

,CLAVE_CATASTRAL,cve_ent,cve_mun,cve_loc,CLAVECATASTRAL_1,curt,notas,mun,zona,manz,CLAVECATASTRAL_2,geometry
0,0240100601,15,024,0000,0240100601000000,194804678009952240860,None,024,01,006,0240100600000000,"POLYGON ((-99.87328 19.80097, -99.87348 19.801..."
1,0240100822,15,024,0000,0240100822000000,194805341109952221128,None,024,01,008,0240100800000000,"POLYGON ((-99.87276 19.80146, -99.87278 19.801..."
2,0240100827,15,024,0000,0240100827000000,194805279409952217160,None,024,01,008,0240100800000000,"POLYGON ((-99.87278 19.80143, -99.87276 19.801..."
3,0240100828,15,024,0000,0240100828000000,194806038509952206157,None,024,01,008,0240100800000000,"POLYGON ((-99.87240 19.80161, -99.87245 19.801..."
4,0240101114,15,024,0000,0240101114000000,194814880409952222562,None,024,01,011,0240101100000000,"POLYGON ((-99.87265 19.80403, -99.87306 19.804..."
...,...,...,...,...,...,...,...,...,...,...,...,...
4307,NaN,NaN,NaN,NaN,0240323900000000,NaN,NaN,024,03,239,0240323900000000,"POLYGON ((-99.92435 19.81447, -99.92433 19.814..."
4308,NaN,NaN,NaN,NaN,0240141000000000,NaN,NaN,024,01,410,0240141000000000,"POLYGON ((-99.86629 19.79529, -99.86647 19.795..."
4309,NaN,NaN,NaN,NaN,0240134000000000,NaN,NaN,024,01,340,0240134000000000,"POLYGON ((-99.86166 19.79180, -99.86237 19.792..."
4310,NaN,NaN,NaN,NaN,0240135200000000,NaN,NaN,024,01,352,0240135200000000,"MULTIPOLYGON (((-99.86557 19.78974, -99.86539 ..."


In [38]:
#Se utiliza esta función para renombrar columna con _1 y eliminar la otra
combine_first_drop(CURT_Man)
CURT_Man

,CLAVE_CATASTRAL,cve_ent,cve_mun,cve_loc,curt,notas,mun,zona,manz,geometry,CLAVECATASTRAL
0,0240100601,15,024,0000,194804678009952240860,None,024,01,006,"POLYGON ((-99.87328 19.80097, -99.87348 19.801...",0240100601000000
1,0240100822,15,024,0000,194805341109952221128,None,024,01,008,"POLYGON ((-99.87276 19.80146, -99.87278 19.801...",0240100822000000
2,0240100827,15,024,0000,194805279409952217160,None,024,01,008,"POLYGON ((-99.87278 19.80143, -99.87276 19.801...",0240100827000000
3,0240100828,15,024,0000,194806038509952206157,None,024,01,008,"POLYGON ((-99.87240 19.80161, -99.87245 19.801...",0240100828000000
4,0240101114,15,024,0000,194814880409952222562,None,024,01,011,"POLYGON ((-99.87265 19.80403, -99.87306 19.804...",0240101114000000
...,...,...,...,...,...,...,...,...,...,...,...
4307,NaN,NaN,NaN,NaN,NaN,NaN,024,03,239,"POLYGON ((-99.92435 19.81447, -99.92433 19.814...",0240323900000000
4308,NaN,NaN,NaN,NaN,NaN,NaN,024,01,410,"POLYGON ((-99.86629 19.79529, -99.86647 19.795...",0240141000000000
4309,NaN,NaN,NaN,NaN,NaN,NaN,024,01,340,"POLYGON ((-99.86166 19.79180, -99.86237 19.792...",0240134000000000
4310,NaN,NaN,NaN,NaN,NaN,NaN,024,01,352,"MULTIPOLYGON (((-99.86557 19.78974, -99.86539 ...",0240135200000000


In [39]:
#Se revisa que coincidan las claves y si hay vacías
CURT_Man[['CLAVECATASTRAL','CLAVE_CATASTRAL']]

,CLAVECATASTRAL,CLAVE_CATASTRAL
0,0240100601000000,0240100601
1,0240100822000000,0240100822
2,0240100827000000,0240100827
3,0240100828000000,0240100828
4,0240101114000000,0240101114
...,...,...
4307,0240323900000000,NaN
4308,0240141000000000,NaN
4309,0240134000000000,NaN
4310,0240135200000000,NaN


In [40]:
#Se vuelve a generar la clave a nivel manzana x las vacías 
separar_digitos(CURT_Man)
CURT_Man

,CLAVE_CATASTRAL,cve_ent,cve_mun,cve_loc,curt,notas,mun,zona,manz,geometry,CLAVECATASTRAL
0,0240100601,15,024,0000,194804678009952240860,None,024,01,006,"POLYGON ((-99.87328 19.80097, -99.87348 19.801...",0240100601000000
1,0240100822,15,024,0000,194805341109952221128,None,024,01,008,"POLYGON ((-99.87276 19.80146, -99.87278 19.801...",0240100822000000
2,0240100827,15,024,0000,194805279409952217160,None,024,01,008,"POLYGON ((-99.87278 19.80143, -99.87276 19.801...",0240100827000000
3,0240100828,15,024,0000,194806038509952206157,None,024,01,008,"POLYGON ((-99.87240 19.80161, -99.87245 19.801...",0240100828000000
4,0240101114,15,024,0000,194814880409952222562,None,024,01,011,"POLYGON ((-99.87265 19.80403, -99.87306 19.804...",0240101114000000
...,...,...,...,...,...,...,...,...,...,...,...
4307,0240323900,NaN,NaN,NaN,NaN,NaN,024,03,239,"POLYGON ((-99.92435 19.81447, -99.92433 19.814...",0240323900000000
4308,0240141000,NaN,NaN,NaN,NaN,NaN,024,01,410,"POLYGON ((-99.86629 19.79529, -99.86647 19.795...",0240141000000000
4309,0240134000,NaN,NaN,NaN,NaN,NaN,024,01,340,"POLYGON ((-99.86166 19.79180, -99.86237 19.792...",0240134000000000
4310,0240135200,NaN,NaN,NaN,NaN,NaN,024,01,352,"MULTIPOLYGON (((-99.86557 19.78974, -99.86539 ...",0240135200000000


In [41]:
#Se vuelve a revisar que coincidan las claves a nivel manzana
CURT_Man[['CLAVECATASTRAL','CLAVE_CATASTRAL']]

,CLAVECATASTRAL,CLAVE_CATASTRAL
0,0240100601000000,0240100601
1,0240100822000000,0240100822
2,0240100827000000,0240100827
3,0240100828000000,0240100828
4,0240101114000000,0240101114
...,...,...
4307,0240323900000000,0240323900
4308,0240141000000000,0240141000
4309,0240134000000000,0240134000
4310,0240135200000000,0240135200


In [42]:
CURT_Man = CURT_Man.loc[~CURT_Man['manz'].astype(str).str.endswith('000')]
CURT_Man.shape

(4305, 11)

In [43]:
#Se revisan columnas 
base_predial_n.columns[base_predial_n.columns.isin(manzanas_mun.columns)]
falta= base_predial_n.columns[~base_predial_n.columns.isin(manzanas_mun.columns)]
falta

Index(['fol_rec', 'nombre', 'rfc', 'ubicacion', 'domicilio_fiscal',
       'sup_terreno', 'sup_construc', 'valor_catastral', 'periodo',
       'periodo_anual', 'fec_pago', 'importe', 'recargos', 'multas',
       'descuento', 'fol_fisc_uuid', 'facturado', 'CLAVE_MANZ', 'CLAVE_PREDIO',
       'CLAVE_CATASTRAL'],
      dtype='object')

In [44]:
cruce_manzana = pd.merge(base_predial_n[['fol_rec', 'nombre', 'rfc', 'ubicacion', 'domicilio_fiscal',
       'sup_terreno', 'sup_construc', 'valor_catastral', 'periodo',
       'periodo_anual', 'fec_pago', 'importe', 'recargos', 'multas',
       'descuento', 'fol_fisc_uuid', 'facturado', 'CLAVE_MANZ', 'CLAVE_PREDIO',
       'CLAVE_CATASTRAL','CLAVECATASTRAL']], CURT_Man, left_on='CLAVE_MANZ', right_on='CLAVE_CATASTRAL', how='inner')
cruce_manzana

,fol_rec,nombre,rfc,ubicacion,domicilio_fiscal,sup_terreno,sup_construc,valor_catastral,periodo,periodo_anual,fec_pago,importe,recargos,multas,descuento,fol_fisc_uuid,facturado,CLAVE_MANZ,CLAVE_PREDIO,CLAVE_CATASTRAL_x,CLAVECATASTRAL_x,CLAVE_CATASTRAL_y,cve_ent,cve_mun,cve_loc,curt,notas,mun,zona,manz,geometry,CLAVECATASTRAL_y
0,IT755056,MEDINA GARCIA ARCELIA*,XAXX010101000,SANTIAGO ACUTZILAPAN,NaN,45.0,26.0,77747.0,DE 01-2022 AL 12-2022,2022,2022-01-03,196.0,NaN,NaN,16.0,1769A2C0-F4B6-407E-9AFA-84D5F1652AD7,NaN,0241101600,0241101614,0241101614,0241101614000000,0241101600,NaN,NaN,NaN,NaN,NaN,024,11,016,"POLYGON ((-99.76502 19.78736, -99.76517 19.787...",0241101600000000
1,IT766308,RUBEN GARCIA PERUSQUIA,XAXX010101000,"AV. BENITO JUAREZ, S/N, SANTIAGO ACUTZILAPAN","AV. BENITO JUAREZ, S/N, SANTIAGO ACUTZILAPAN",166.0,NaN,74593.0,DE 01-2022 AL 12-2022,2022,2022-01-05,195.0,NaN,NaN,25.0,30D5C561-10C1-44A4-BBEE-0B643F5349CF,NaN,0241101600,0241101653,0241101653,0241101653000000,0241101600,NaN,NaN,NaN,NaN,NaN,024,11,016,"POLYGON ((-99.76502 19.78736, -99.76517 19.787...",0241101600000000
2,TM005017,GUSTAVO ORDOÑEZ MEDINA,XAXX010101000,"BENITO JUAREZ, S/N, SANTIAGO ACUTZILAPAN","BENITO JUAREZ, S/N, SANTIAGO ACUTZILAPAN",335.0,NaN,157450.0,DE 01-2022 AL 12-2022,2022,2022-01-10,255.0,NaN,NaN,40.0,4D084509-EBA6-45FD-8CAF-F32A0F6F64BF,NaN,0241101600,0241101652,0241101652,0241101652000000,0241101600,NaN,NaN,NaN,NaN,NaN,024,11,016,"POLYGON ((-99.76502 19.78736, -99.76517 19.787...",0241101600000000
3,TM004753,GARCIA JIMENEZ RUBEN,XAXX010101000,"AV. BENITO JUAREZ, S/N, SANTIAGO ACUTZILAPAN",NaN,166.0,NaN,56051.0,DE 01-2017 AL 12-2017,2017,2022-01-19,189.0,94.0,NaN,NaN,821C0E7C-5017-43DD-AD9A-8AFA390473E8,NaN,0241101600,0241101662,0241101662,0241101662000000,0241101600,NaN,NaN,NaN,NaN,NaN,024,11,016,"POLYGON ((-99.76502 19.78736, -99.76517 19.787...",0241101600000000
4,TM006685,ZALDIVAR GONZALEZ JOSE TRINIDAD *,XAXX010101000,CALLE JUAREZ SANTIAGO ACUTZILAPAN,NaN,100.0,40.0,145682.0,DE 01-2022 AL 12-2022,2022,2022-01-24,218.0,NaN,NaN,48.0,EEE49BF2-7ACF-4F9B-A5FC-ABF235B4CCF9,NaN,0241101600,0241101634,0241101634,0241101634000000,0241101600,NaN,NaN,NaN,NaN,NaN,024,11,016,"POLYGON ((-99.76502 19.78736, -99.76517 19.787...",0241101600000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14037,TM051701,GUTIERREZ MIRA EPITACIA CONCEPCION,XAXX010101000,"CALLE 5 DE MAYO, S/N, CUENDO",NaN,657.0,NaN,12155.0,DE 01-2022 AL 12-2022,2022,2022-11-29,200.0,30.0,NaN,18.0,FC602273-F2CE-4C21-B534-5A94502EB1C4,NaN,0240405200,0240405203,0240405203,0240405203000000,0240405200,NaN,NaN,NaN,NaN,NaN,024,04,052,"POLYGON ((-99.95240 19.80050, -99.95259 19.800...",0240405200000000
14038,TM055634,EJIDO SAN JOSE DEL TUNAL*,XAXX010101000,EJIDO SAN JOSE DEL TUNAL,NaN,1907980.0,NaN,3110007.0,DE 01-2017 AL 12-2017,2017,2022-12-20,2838.0,2838.0,1986.0,4824.0,0D126631-F019-48EB-8429-03BEF09435B9,NaN,0241525400,0241525401,0241525401,0241525401000000,0241525400,NaN,NaN,NaN,NaN,NaN,024,15,254,"POLYGON ((-99.81669 19.83143, -99.81634 19.831...",0241525400000000
14039,TM055636,EJIDO SAN JOSE DEL TUNAL*,XAXX010101000,EJIDO SAN JOSE DEL TUNAL,NaN,1572276.0,NaN,11383278.0,DE 01-2017 AL 12-2017,2017,2022-12-20,3240.0,1620.0,NaN,1620.0,0D126631-F019-48EB-8429-03BEF09435B9,NaN,0240407400,0240407401,0240407401,0240407401000000,0240407400,NaN,NaN,NaN,NaN,NaN,024,04,074,"POLYGON ((-99.95390 19.83083, -99.95288 19.830...",0240407400000000
14040,TM049657,GARCIA VALVERDE PATRICIA,XAXX010101000,"CAMINO SANTA CRUZ, S/N, LA PALMA",NaN,579.0,NaN,115943.0,DE 01-2022 AL 12-2022,2022,2022-12-28,240.0,40.0,NaN,NaN,3A6E68CF-C68C-4295-AEE7-253BA383E5F7,NaN,0240721200,0240721201,0240721201,0240721201000000,0240721200,NaN,NaN,NaN,NaN,NaN,024,07,212,"POLYGON ((-99.87280 19.81319, -99.87388 19.813...",0240721200000000


In [45]:
cruce_manzana.columns

Index(['fol_rec', 'nombre', 'rfc', 'ubicacion', 'domicilio_fiscal',
       'sup_terreno', 'sup_construc', 'valor_catastral', 'periodo',
       'periodo_anual', 'fec_pago', 'importe', 'recargos', 'multas',
       'descuento', 'fol_fisc_uuid', 'facturado', 'CLAVE_MANZ', 'CLAVE_PREDIO',
       'CLAVE_CATASTRAL_x', 'CLAVECATASTRAL_x', 'CLAVE_CATASTRAL_y', 'cve_ent',
       'cve_mun', 'cve_loc', 'curt', 'notas', 'mun', 'zona', 'manz',
       'geometry', 'CLAVECATASTRAL_y'],
      dtype='object')

In [46]:
#Se vuelve a revisar que coincidan las claves para decidir a cuál darle prioridad
cruce_manzana[['CLAVECATASTRAL_x','CLAVECATASTRAL_y','CLAVE_CATASTRAL_x','CLAVE_CATASTRAL_y']]

,CLAVECATASTRAL_x,CLAVECATASTRAL_y,CLAVE_CATASTRAL_x,CLAVE_CATASTRAL_y
0,0241101614000000,0241101600000000,0241101614,0241101600
1,0241101653000000,0241101600000000,0241101653,0241101600
2,0241101652000000,0241101600000000,0241101652,0241101600
3,0241101662000000,0241101600000000,0241101662,0241101600
4,0241101634000000,0241101600000000,0241101634,0241101600
...,...,...,...,...
14037,0240405203000000,0240405200000000,0240405203,0240405200
14038,0241525401000000,0241525400000000,0241525401,0241525400
14039,0240407401000000,0240407400000000,0240407401,0240407400
14040,0240721201000000,0240721200000000,0240721201,0240721200


In [47]:
cruce_manzana['CLAVECATASTRAL']=cruce_manzana.CLAVECATASTRAL_x.combine_first(cruce_manzana.CLAVECATASTRAL_y)
cruce_manzana[cruce_manzana.CLAVECATASTRAL.isna()]
#Eliminar columna de CLAVECATASTRAL_y y renombrar columna de CLAVECATASTRAL_x
cruce_manzana.drop(['CLAVECATASTRAL_y','CLAVECATASTRAL_x'],axis=1,inplace=True )
cruce_manzana.columns

Index(['fol_rec', 'nombre', 'rfc', 'ubicacion', 'domicilio_fiscal',
       'sup_terreno', 'sup_construc', 'valor_catastral', 'periodo',
       'periodo_anual', 'fec_pago', 'importe', 'recargos', 'multas',
       'descuento', 'fol_fisc_uuid', 'facturado', 'CLAVE_MANZ', 'CLAVE_PREDIO',
       'CLAVE_CATASTRAL_x', 'CLAVE_CATASTRAL_y', 'cve_ent', 'cve_mun',
       'cve_loc', 'curt', 'notas', 'mun', 'zona', 'manz', 'geometry',
       'CLAVECATASTRAL'],
      dtype='object')

In [48]:
cruce_manzana['CLAVE_CATASTRAL']=cruce_manzana.CLAVE_CATASTRAL_x.combine_first(cruce_manzana.CLAVE_CATASTRAL_y)
# cruce_manzana[cruce_manzana.CLAVE_CATASTRAL.isna()]
#Eliminar columna de CLAVECATASTRAL_y y renombrar columna de CLAVECATASTRAL_x
cruce_manzana.drop(['CLAVE_CATASTRAL_y','CLAVE_CATASTRAL_x'],axis=1,inplace=True )
cruce_manzana.columns

Index(['fol_rec', 'nombre', 'rfc', 'ubicacion', 'domicilio_fiscal',
       'sup_terreno', 'sup_construc', 'valor_catastral', 'periodo',
       'periodo_anual', 'fec_pago', 'importe', 'recargos', 'multas',
       'descuento', 'fol_fisc_uuid', 'facturado', 'CLAVE_MANZ', 'CLAVE_PREDIO',
       'cve_ent', 'cve_mun', 'cve_loc', 'curt', 'notas', 'mun', 'zona', 'manz',
       'geometry', 'CLAVECATASTRAL', 'CLAVE_CATASTRAL'],
      dtype='object')

In [49]:
cruce_manzana[['CLAVECATASTRAL','CLAVE_CATASTRAL']]

,CLAVECATASTRAL,CLAVE_CATASTRAL
0,0241101614000000,0241101614
1,0241101653000000,0241101653
2,0241101652000000,0241101652
3,0241101662000000,0241101662
4,0241101634000000,0241101634
...,...,...
14037,0240405203000000,0240405203
14038,0241525401000000,0241525401
14039,0240407401000000,0240407401
14040,0240721201000000,0240721201


In [50]:
len(cruce_manzana.CLAVECATASTRAL.unique())

13992

In [51]:
#Base de los registros que no cruzaron x curt a nivel predio ni manzana
base_predial_nm =base_predial_n.loc[~base_predial_n['CLAVE_MANZ'].isin(cruce_manzana['CLAVE_MANZ'])]
base_predial_nm

,fol_rec,nombre,rfc,CLAVECATASTRAL,ubicacion,domicilio_fiscal,sup_terreno,sup_construc,valor_catastral,periodo,periodo_anual,fec_pago,importe,recargos,multas,descuento,fol_fisc_uuid,facturado,CLAVE_MANZ,CLAVE_PREDIO,CLAVE_CATASTRAL
294,IT755189,SANCHEZ ARANDA MARIA ANGELICA,XAXX010101000,0240153712010032,A SAN LORENZO SANTA CRUZ BOMBATEVI,NaN,130.00,75.0,501066.0,DE 01-2022 AL 12-2022,2022,2022-01-03,670.00,NaN,NaN,108.00,1769A2C0-F4B6-407E-9AFA-84D5F1652AD7,NaN,0240153700,0240153712,0240153712
1173,IT766353,CARDENAS MONTIEL SERGIO BERNARDO,XAXX010101000,0240128013250004,"VILLADA VICENTE OTE # S/N - 04, COL. MORELOS, ...",NaN,183.00,177.0,584774.0,DE 01-2022 AL 12-2022,2022,2022-01-05,800.00,NaN,NaN,128.00,30D5C561-10C1-44A4-BBEE-0B643F5349CF,NaN,0240128000,0240128013,0240128013
1177,IT766355,SERGIO BERNARDO CARDENAS MONTIEL,XAXX010101000,0240128014000000,"RUIZ Y FLORES MAXIMINO, S/N, COLONIA FELIPE UREÑA","RUIZ Y FLORES MAXIMINO, S/N, COLONIA FELIPE UREÑA",327.00,233.0,1123731.0,DE 01-2022 AL 12-2022,2022,2022-01-05,1998.00,NaN,NaN,678.00,30D5C561-10C1-44A4-BBEE-0B643F5349CF,NaN,0240128000,0240128014,0240128014
1314,IT769198,RIVERA PEDRAZA JUANA MARIA ESTHER*,XAXX010101000,0240153712010031,A SAN LORENZO SANTA CRUZ BOMBATEVI,NaN,130.00,75.0,500884.0,DE 01-2022 AL 12-2022,2022,2022-01-05,670.00,NaN,NaN,54.00,30D5C561-10C1-44A4-BBEE-0B643F5349CF,NaN,0240153700,0240153712,0240153712
1829,IT759305,RUBEN CRESPO CRUZ Y DALIA MATEOS DE JESUS,XAXX010101000,0240153712010036,A SAN LORENZO SANTA CRUZ BOMBATEVI,NaN,130.00,75.0,500884.0,DE 01-2022 AL 12-2022,2022,2022-01-06,670.00,NaN,NaN,108.00,61F6388E-AAEC-4755-83A4-FE76FF7197C8,NaN,0240153700,0240153712,0240153712
1946,IT773650,LIMA ESTEVEZ MANUEL Y MARIA PAULA ALCANTARA ...,NaN,0240153712010053,A SAN LORENZO SANTA CRUZ BOMBATEVI,NaN,130.00,75.0,500884.0,DE 01-2022 AL 06-2022,2022,2022-01-06,335.14,NaN,NaN,53.60,NaN,S,0240153700,0240153712,0240153712
2497,IT769593,GARCIA MARTINEZ FERNANDO,XAXX010101000,0240153712010055,A SAN LORENZO 55 SANTA CRUZ BOMBATEVI,NaN,130.00,75.0,500884.0,DE 01-2022 AL 12-2022,2022,2022-01-07,670.00,NaN,NaN,108.00,64919EDE-3867-4F6B-AF23-11ADE32C6B78,NaN,0240153700,0240153712,0240153712
2744,IT773772,LORENZO SIMON IVAN Y MARICARMEN ROSAS CRUZ,NaN,0240153712010020,A SAN LORENZO 20 SANTA CRUZ BOMBATEVI.,NaN,130.00,75.0,481131.0,DE 01-2021 AL 12-2021,2021,2022-01-07,642.51,106.97,NaN,NaN,NaN,S,0240153700,0240153712,0240153712
2859,IT769759,ROBERTO COLIN MARTINEZ,XAXX010101000,0240128021000000,RUIZ Y FLORES MAXIMINO NTE. S.N.,NaN,182.00,NaN,98280.0,DE 01-2022 AL 12-2022,2022,2022-01-08,203.00,NaN,NaN,32.00,4D3810DB-DBB6-4D0C-AB51-E529BA35F6C7,NaN,0240128000,0240128021,0240128021
2918,TM000021,RENE PEREZ GONZALEZ,XAXX010101000,0241113004000000,SANTIAGO ACUTZILAPAN,NaN,948.00,NaN,381840.0,DE 01-2022 AL 12-2022,2022,2022-01-10,578.00,NaN,NaN,46.00,4D084509-EBA6-45FD-8CAF-F32A0F6F64BF,NaN,0241113000,0241113004,0241113004


In [52]:
#Se concatenan el cruce 2 con el cruce 3
cruce_predial_manz = pd.concat([cruce_cat,cruce_manzana])
cruce_predial_manz

,fol_rec,nombre,rfc,ubicacion,domicilio_fiscal,sup_terreno,sup_construc,valor_catastral,periodo,periodo_anual,fec_pago,importe,recargos,multas,descuento,fol_fisc_uuid,facturado,CLAVE_MANZ,CLAVE_PREDIO,geometry,cve_ent,cve_mun,cve_loc,curt,notas,CLAVE_CATASTRAL,CLAVECATASTRAL,mun,zona,manz
0,IT765964,ALFONSO SANCHEZ VERA,XAXX010101000,"MANUEL M. PONCE, S/N, COLONIA LAS FUENTES","MANUEL M. PONCE, S/N ,COL. LAS FUENTES",172.0,NaN,148771.0,DE 01-2022 AL 12-2022,2022,2022-01-03,219.0,NaN,NaN,18.0,1769A2C0-F4B6-407E-9AFA-84D5F1652AD7,NaN,0240127100,0240127101,"MULTIPOLYGON (((-99.88402 19.80165, -99.88415 ...",15,024,0000,None,14 _ Ver curtnotas,0240127101,02401271010F0014,NaN,NaN,NaN
1,IT765966,DURAN MARTINEZ ELVIRA,XAXX010101000,"MANUEL M. PONCE, S/N, COL. LAS FUENTES","MANUEL M. PONCE, S/N, COL. LAS FUENTES",172.0,NaN,148771.0,DE 01-2022 AL 12-2022,2022,2022-01-03,219.0,NaN,NaN,18.0,1769A2C0-F4B6-407E-9AFA-84D5F1652AD7,NaN,0240127100,0240127101,"MULTIPOLYGON (((-99.88402 19.80165, -99.88415 ...",15,024,0000,None,14 _ Ver curtnotas,0240127101,02401271010F0012,NaN,NaN,NaN
2,TM000028,ASOCIACION CIVIL DE MAESTROS X UNA VIVIENDA DI...,XAXX010101000,MANUEL M. PONCE 36 INT. 02 LAS FUENTES,NaN,172.0,156.0,741760.0,DE 01-2022 AL 12-2022,2022,2022-01-10,1080.0,NaN,NaN,168.0,4D084509-EBA6-45FD-8CAF-F32A0F6F64BF,NaN,0240127100,0240127101,"MULTIPOLYGON (((-99.88402 19.80165, -99.88415 ...",15,024,0000,None,14 _ Ver curtnotas,0240127101,02401271010F0002,NaN,NaN,NaN
3,TM000582,MARTINEZ HERRERA ROSA*,XAXX010101000,MANUEL M. PONCE 36 INT 20,NaN,172.0,NaN,148780.0,DE 01-2022 AL 12-2022,2022,2022-01-12,219.0,NaN,NaN,36.0,9140689A-9EBD-4A6E-A77D-18C1360BD6E3,NaN,0240127100,0240127101,"MULTIPOLYGON (((-99.88402 19.80165, -99.88415 ...",15,024,0000,None,14 _ Ver curtnotas,0240127101,02401271010F0018,NaN,NaN,NaN
4,TM005563,BRAULIO MARTINEZ GERMAN,XAXX010101000,"MANUEL M. PONCE # S/N - 30, COL. MIGUEL PORTIL...",NaN,172.0,48.0,319658.0,DE 01-2022 AL 12-2022,2022,2022-01-13,417.0,NaN,NaN,66.0,7F151F69-731B-4D79-BD11-29C697560F17,NaN,0240127100,0240127101,"MULTIPOLYGON (((-99.88402 19.80165, -99.88415 ...",15,024,0000,None,14 _ Ver curtnotas,0240127101,02401271010F0030,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14037,TM051701,GUTIERREZ MIRA EPITACIA CONCEPCION,XAXX010101000,"CALLE 5 DE MAYO, S/N, CUENDO",NaN,657.0,NaN,12155.0,DE 01-2022 AL 12-2022,2022,2022-11-29,200.0,30.0,NaN,18.0,FC602273-F2CE-4C21-B534-5A94502EB1C4,NaN,0240405200,0240405203,"POLYGON ((-99.95240 19.80050, -99.95259 19.800...",NaN,NaN,NaN,NaN,NaN,0240405203,0240405203000000,024,04,052
14038,TM055634,EJIDO SAN JOSE DEL TUNAL*,XAXX010101000,EJIDO SAN JOSE DEL TUNAL,NaN,1907980.0,NaN,3110007.0,DE 01-2017 AL 12-2017,2017,2022-12-20,2838.0,2838.0,1986.0,4824.0,0D126631-F019-48EB-8429-03BEF09435B9,NaN,0241525400,0241525401,"POLYGON ((-99.81669 19.83143, -99.81634 19.831...",NaN,NaN,NaN,NaN,NaN,0241525401,0241525401000000,024,15,254
14039,TM055636,EJIDO SAN JOSE DEL TUNAL*,XAXX010101000,EJIDO SAN JOSE DEL TUNAL,NaN,1572276.0,NaN,11383278.0,DE 01-2017 AL 12-2017,2017,2022-12-20,3240.0,1620.0,NaN,1620.0,0D126631-F019-48EB-8429-03BEF09435B9,NaN,0240407400,0240407401,"POLYGON ((-99.95390 19.83083, -99.95288 19.830...",NaN,NaN,NaN,NaN,NaN,0240407401,0240407401000000,024,04,074
14040,TM049657,GARCIA VALVERDE PATRICIA,XAXX010101000,"CAMINO SANTA CRUZ, S/N, LA PALMA",NaN,579.0,NaN,115943.0,DE 01-2022 AL 12-2022,2022,2022-12-28,240.0,40.0,NaN,NaN,3A6E68CF-C68C-4295-AEE7-253BA383E5F7,NaN,0240721200,0240721201,"POLYGON ((-99.87280 19.81319, -99.87388 19.813...",NaN,NaN,NaN,NaN,NaN,0240721201,0240721201000000,024,07,212


In [53]:
cruce_predial_manz[['CLAVECATASTRAL','CLAVE_CATASTRAL']]

,CLAVECATASTRAL,CLAVE_CATASTRAL
0,02401271010F0014,0240127101
1,02401271010F0012,0240127101
2,02401271010F0002,0240127101
3,02401271010F0018,0240127101
4,02401271010F0030,0240127101
...,...,...
14037,0240405203000000,0240405203
14038,0241525401000000,0241525401
14039,0240407401000000,0240407401
14040,0240721201000000,0240721201


In [54]:
cruce_predial_manz.columns

Index(['fol_rec', 'nombre', 'rfc', 'ubicacion', 'domicilio_fiscal',
       'sup_terreno', 'sup_construc', 'valor_catastral', 'periodo',
       'periodo_anual', 'fec_pago', 'importe', 'recargos', 'multas',
       'descuento', 'fol_fisc_uuid', 'facturado', 'CLAVE_MANZ', 'CLAVE_PREDIO',
       'geometry', 'cve_ent', 'cve_mun', 'cve_loc', 'curt', 'notas',
       'CLAVE_CATASTRAL', 'CLAVECATASTRAL', 'mun', 'zona', 'manz'],
      dtype='object')

In [55]:
#Se concatena el Cruce 1 con el concatenado del Cruce 2 y 3
cruce_predial_completo = pd.concat([cruce_predial,cruce_predial_manz])
cruce_predial_completo

,fol_rec,nombre,rfc,ubicacion,domicilio_fiscal,sup_terreno,sup_construc,valor_catastral,periodo,periodo_anual,fec_pago,importe,recargos,multas,descuento,fol_fisc_uuid,facturado,CLAVE_MANZ,CLAVE_PREDIO,CLAVE_CATASTRAL,CLAVECATASTRAL,cve_ent,cve_mun,cve_loc,curt,notas,geometry,mun,zona,manz
0,IT758593,LEONARDO GALLEGOS SANCHEZ,XAXX010101000,"SIN NOMBRE, SAN LORENZO TLACOTEPEC","SIN NOMBRE, SAN LORENZO TLACOTEPEC",533.0,NaN,61295.0,DE 01-2022 AL 12-2022,2022,2022-01-03,219.0,NaN,NaN,18.0,1769A2C0-F4B6-407E-9AFA-84D5F1652AD7,NaN,0240325200,0240325221,0240325221,0240325221000000,15,024,0000,194917088909954124912,NaN,"POLYGON ((-99.90357 19.82164, -99.90345 19.821...",NaN,NaN,NaN
1,IT765967,HERNADEZ GUZMAN MAYOLO*,XAXX010101000,LOS ANGELES SANTA CRUZ BOMBATEVI,NaN,915.0,30.0,475277.0,DE 01-2022 AL 12-2022,2022,2022-01-03,634.0,NaN,NaN,50.0,1769A2C0-F4B6-407E-9AFA-84D5F1652AD7,NaN,0240152300,0240152390,0240152390,0240152390000000,15,024,0000,194822721709953147754,NaN,"POLYGON ((-99.88728 19.80646, -99.88734 19.806...",NaN,NaN,NaN
2,IT758595,JACINTO NOVERON SANTANA,XAXX010101000,"JORGE JIMENEZ CANTU, SANTA CRUZ BOMBATEVI","JORGE JIMENEZ CANTU, SANTA CRUZ BOMBATEVI",468.0,300.0,1040558.0,DE 01-2022 AL 12-2022,2022,2022-01-03,1776.0,NaN,NaN,144.0,1769A2C0-F4B6-407E-9AFA-84D5F1652AD7,NaN,0240155600,0240155615,0240155615,0240155615000000,15,024,0000,194758461709953209553,NaN,"POLYGON ((-99.88911 19.79972, -99.88906 19.799...",NaN,NaN,NaN
3,IT755065,IVONNE VIOLETA GARCIA MADARIAGA,XAXX010101000,"LAZARO CARDENAS DEL RIO, S/N, COL. LA ASCENSION","LAZARO CARDENAS DEL RIO, S/N, COL. LA ASCENSION",162.0,96.0,556985.0,DE 01-2022 AL 12-2022,2022,2022-01-03,750.0,NaN,NaN,120.0,1769A2C0-F4B6-407E-9AFA-84D5F1652AD7,NaN,0240145200,0240145211,0240145211,0240145211000000,15,024,0000,194733334909952394393,NaN,"POLYGON ((-99.87755 19.79269, -99.87761 19.792...",NaN,NaN,NaN
4,IT755070,NAVA ALBARRAN MARICELA GRACIA*,XAXX010101000,"CALLE NOCHE TRISTE NUM. 11, COL. EL CALVARIO","CALLE NOCHE TRISTE NUM. 11, COL. EL CALVARIO",452.0,204.0,1240253.0,DE 01-2022 AL 12-2022,2022,2022-01-03,2346.0,NaN,NaN,798.0,1769A2C0-F4B6-407E-9AFA-84D5F1652AD7,NaN,0240104100,0240104121,0240104121,0240104121000000,15,024,0000,194749888209952194264,NaN,"POLYGON ((-99.87195 19.79725, -99.87200 19.797...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14037,TM051701,GUTIERREZ MIRA EPITACIA CONCEPCION,XAXX010101000,"CALLE 5 DE MAYO, S/N, CUENDO",NaN,657.0,NaN,12155.0,DE 01-2022 AL 12-2022,2022,2022-11-29,200.0,30.0,NaN,18.0,FC602273-F2CE-4C21-B534-5A94502EB1C4,NaN,0240405200,0240405203,0240405203,0240405203000000,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-99.95240 19.80050, -99.95259 19.800...",024,04,052
14038,TM055634,EJIDO SAN JOSE DEL TUNAL*,XAXX010101000,EJIDO SAN JOSE DEL TUNAL,NaN,1907980.0,NaN,3110007.0,DE 01-2017 AL 12-2017,2017,2022-12-20,2838.0,2838.0,1986.0,4824.0,0D126631-F019-48EB-8429-03BEF09435B9,NaN,0241525400,0241525401,0241525401,0241525401000000,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-99.81669 19.83143, -99.81634 19.831...",024,15,254
14039,TM055636,EJIDO SAN JOSE DEL TUNAL*,XAXX010101000,EJIDO SAN JOSE DEL TUNAL,NaN,1572276.0,NaN,11383278.0,DE 01-2017 AL 12-2017,2017,2022-12-20,3240.0,1620.0,NaN,1620.0,0D126631-F019-48EB-8429-03BEF09435B9,NaN,0240407400,0240407401,0240407401,0240407401000000,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-99.95390 19.83083, -99.95288 19.830...",024,04,074
14040,TM049657,GARCIA VALVERDE PATRICIA,XAXX010101000,"CAMINO SANTA CRUZ, S/N, LA PALMA",NaN,579.0,NaN,115943.0,DE 01-2022 AL 12-2022,2022,2022-12-28,240.0,40.0,NaN,NaN,3A6E68CF-C68C-4295-AEE7-253BA383E5F7,NaN,0240721200,0240721201,0240721201,0240721201000000,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-99.87280 19.81319, -99.87388 19.813...",024,07,212


In [56]:
cruce_predial_completo.CLAVECATASTRAL.value_counts()

0240801089000000    3
0240165026000000    2
0240165061000000    2
0240165003000000    2
0240165013000000    2
                   ..
0240113514000000    1
0240113537000000    1
0240113502000000    1
0240113508000000    1
0240721202000000    1
Name: CLAVECATASTRAL, Length: 15685, dtype: int64

In [57]:
len(cruce_predial_completo.CLAVECATASTRAL.unique())

15685

In [58]:
cruce_predial_completo.columns

Index(['fol_rec', 'nombre', 'rfc', 'ubicacion', 'domicilio_fiscal',
       'sup_terreno', 'sup_construc', 'valor_catastral', 'periodo',
       'periodo_anual', 'fec_pago', 'importe', 'recargos', 'multas',
       'descuento', 'fol_fisc_uuid', 'facturado', 'CLAVE_MANZ', 'CLAVE_PREDIO',
       'CLAVE_CATASTRAL', 'CLAVECATASTRAL', 'cve_ent', 'cve_mun', 'cve_loc',
       'curt', 'notas', 'geometry', 'mun', 'zona', 'manz'],
      dtype='object')

In [78]:
#Guardar base de predial vs curt
cruce_predial_completo.to_csv(r"I:\Mi unidad\Cruces_predial_otros\Atlacomulco\Bases_salida_predial/Atlacomulco_cruce_predialxcurt.csv")

Cruce de predial OD con CURT vs Predial(Predial, deudores y propietarios)

In [59]:
#Base predial
pred_pred.tail(3)

,CLAVECATASTRAL,CLAVEASENTAMIENTO,CLAVECP,CLAVEMUNICIPIO,CLAVEENTIDAD,CLAVEMANZANA,CLAVEZONA,CLAVESTATUS,DIRECCION,VALORTERRENO,VALORTERRENOCOMUN,VALORCONSTPROP,VALORCONSTCOMUN,SUPERFICIE,SUPERFICIETERRCOMUN,SUPERFICIECONST,SUPERFICIECONSTCOMUN,CLAVEANTERIOR,LATITUD,LONGITUD,FECHAALTA,ASENTAMIENTO_NR,CLAVECP_NR
29755,0241530823000000,555,50475,14,15,308,15,1,SIN NOMBRE S/N S/N,89910,0,0,0,810,0,0,0,0241530807000000,0,0,2016-03-04 15:23:03.0,,
29756,0241530824000000,554,50475,14,15,308,15,1,SIN NOMBRE S/N S/N,568004,0,0,0,9986,0,0,0,0241580410000000,0,0,2016-03-23 11:04:24.0,,
29757,0241530825000000,555,50475,14,15,308,15,1,Sin Nombre S/N,16095,0,0,0,145,0,0,0,,0,0,2017-05-03 18:06:18.0,,


In [60]:
#Base propietarios
pred_prop.tail(3)

,NOMBRE,APATERNO,AMATERNO,RFC,CLAVECP,CLAVEMUNICIPIO,CLAVEENTIDAD,CALLE,CLAVECATASTRAL,CLAVESTATUS,CLAVEASENTAMIENTO,EMAIL,CURP,ASENTAMIENTO_NR,CLAVECP_NR
29755,MIRIAM,MELCHOR,DE LA CRUZ,,50475,14,15,SIN NOMBRE S/N S/N,0241530823000000,1,555,,,,
29756,FELIPE,HIPOLITO,RUIZ,,50475,14,15,SIN NOMBRE S/N S/N,0241530824000000,1,554,,,,
29757,ORDOÑEZ MENDOZA JUSTO,,,,50475,14,15,"CALLE SIN NOMBRE,# S/N,COL.SAN LUIS BORO,CP. 5...",0241530825000000,1,555,,,,


In [61]:
#Base deudores
pred_deu.tail(3)

,CLAVECATASTRAL,EJERCICIO,CLAVEPERIODOINI,CLAVEPERIODOFIN,VALORCATASTRAL,STATUS
260989,0241530825000000,2018,1,12,16095,Adeudo
260990,0241530825000000,2019,1,12,17690,Adeudo
260991,0241530825000000,2021,1,12,21025,Adeudo


In [62]:
#borrar columnas vacias
delete_empty(pred_prop)
delete_empty(pred_pred)
delete_empty(pred_deu)

,CLAVECATASTRAL,EJERCICIO,CLAVEPERIODOINI,CLAVEPERIODOFIN,VALORCATASTRAL,STATUS
0,0240000000PRUEBA,2018.0,1.0,12.0,245321.0,Adeudo
1,0240000000PRUEBA,2019.0,1.0,12.0,273408.0,Adeudo
2,0240000001PRUEBA,2012.0,1.0,12.0,22222.0,Adeudo
3,0240000001PRUEBA,2013.0,1.0,12.0,2222.0,Adeudo
4,0240000004PRUEBA,2011.0,1.0,12.0,400000.0,Adeudo
...,...,...,...,...,...,...
260987,0241530824000000,2019.0,1.0,12.0,624325.0,Pago Municipal
260988,0241530824000000,2021.0,1.0,12.0,742060.0,Adeudo
260989,0241530825000000,2018.0,1.0,12.0,16095.0,Adeudo
260990,0241530825000000,2019.0,1.0,12.0,17690.0,Adeudo


In [63]:
#borrar duplicados de las bases x CLAVECATASTRAL
pred_pred.drop_duplicates(subset='CLAVECATASTRAL', inplace=True)
pred_prop.drop_duplicates(subset='CLAVECATASTRAL', inplace=True)

In [64]:
pred_deu.columns

Index(['CLAVECATASTRAL', 'EJERCICIO', 'CLAVEPERIODOINI', 'CLAVEPERIODOFIN',
       'VALORCATASTRAL', 'STATUS'],
      dtype='object')

In [65]:
#Se borran duplicados de la base deudores x CC y después se ordenan x Ejercicio Fiscal
pred_deu.drop_duplicates(subset='CLAVECATASTRAL',keep='first',inplace=True)
pred_deu.sort_values(by='EJERCICIO',ascending=False,inplace=True)

In [66]:
#Se crea la CC a nivel predio de cada base
pred_pred['CLAVE_CATASTRAL']= pred_pred.CLAVECATASTRAL.astype(str).str[0:10]
pred_prop['CLAVE_CATASTRAL']= pred_prop.CLAVECATASTRAL.astype(str).str[0:10]
pred_deu['CLAVE_CATASTRAL']= pred_deu.CLAVECATASTRAL.astype(str).str[0:10]

In [67]:
#Asegurarse que se cortaron todas las claves y no hay nan (Descomentar una x una)
pred_pred[pred_pred.CLAVE_CATASTRAL.isna()]
# pred_prop[pred_prop.CLAVE_CATASTRAL.isna()]
# pred_deu[pred_deu.CLAVE_CATASTRAL.isna()]

,CLAVECATASTRAL,CLAVEASENTAMIENTO,CLAVECP,CLAVEMUNICIPIO,CLAVEENTIDAD,CLAVEMANZANA,CLAVEZONA,CLAVESTATUS,DIRECCION,VALORTERRENO,VALORTERRENOCOMUN,VALORCONSTPROP,VALORCONSTCOMUN,SUPERFICIE,SUPERFICIETERRCOMUN,SUPERFICIECONST,SUPERFICIECONSTCOMUN,CLAVEANTERIOR,FECHAALTA,CLAVE_CATASTRAL


In [68]:
#Qué columnas están dentro de mi base x cruzar, que no están dentro de la 1ra base 
pred_pred.columns[pred_pred.columns.isin(pred_prop.columns)]
falta= pred_pred.columns[~pred_pred.columns.isin(pred_prop.columns)]
falta

Index(['CLAVEMANZANA', 'CLAVEZONA', 'DIRECCION', 'VALORTERRENO',
       'VALORTERRENOCOMUN', 'VALORCONSTPROP', 'VALORCONSTCOMUN', 'SUPERFICIE',
       'SUPERFICIETERRCOMUN', 'SUPERFICIECONST', 'SUPERFICIECONSTCOMUN',
       'CLAVEANTERIOR', 'FECHAALTA'],
      dtype='object')

Cruce 4: Predial - Predial vs Predial - Propietarios

In [69]:
#Cruce de predial general con propietario (el resultado de este cruce es la primera parte de la base completa de predial, no debe tener duplicados por clave catastral unica a 16 digitos y esa es la key de cruce)
predial_general = pd.merge(pred_pred[['CLAVEMANZANA', 'CLAVEZONA', 'DIRECCION', 'VALORTERRENO',
       'VALORTERRENOCOMUN', 'VALORCONSTPROP', 'VALORCONSTCOMUN', 'SUPERFICIE',
       'SUPERFICIETERRCOMUN', 'SUPERFICIECONST', 'SUPERFICIECONSTCOMUN',
       'CLAVEANTERIOR', 'FECHAALTA','CLAVE_CATASTRAL','CLAVECATASTRAL']], pred_prop, on="CLAVECATASTRAL", how="inner")
predial_general

,CLAVEMANZANA,CLAVEZONA,DIRECCION,VALORTERRENO,VALORTERRENOCOMUN,VALORCONSTPROP,VALORCONSTCOMUN,SUPERFICIE,SUPERFICIETERRCOMUN,SUPERFICIECONST,SUPERFICIECONSTCOMUN,CLAVEANTERIOR,FECHAALTA,CLAVE_CATASTRAL_x,CLAVECATASTRAL,NOMBRE,APATERNO,AMATERNO,RFC,CLAVECP,CLAVEMUNICIPIO,CLAVEENTIDAD,CALLE,CLAVESTATUS,CLAVEASENTAMIENTO,EMAIL,CURP,ASENTAMIENTO_NR,CLAVECP_NR,CLAVE_CATASTRAL_y
0,NaN,NaN,PRUEBA 1 1,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,2013-09-25 12:59:34.0,0240000000,0240000000PRUEBA,PRUEBA 1,NaN,NaN,NaN,51260.0,7.0,15.0,PRUEBA 1,1.0,1054.0,NaN,NaN,NaN,NaN,0240000000
1,NaN,NaN,PRUEBA 2 2,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,NaN,2013-09-25 13:33:11.0,0240000001,0240000001PRUEBA,PRUEBA 2,NaN,NaN,NaN,56766.0,17.0,15.0,PRUEBA 2 2,1.0,9282.0,NaN,NaN,NaN,NaN,0240000001
2,NaN,NaN,PRUEBA 111 111,300000,NaN,145000,NaN,100.0,NaN,200.0,NaN,NaN,2013-10-10 10:47:42.0,0240000004,0240000004PRUEBA,PRUEBA 1111,PRUEBA,PRUEBA,NaN,52542.0,6.0,15.0,PRUEBA 111 111,1.0,6881.0,NaN,NaN,NaN,NaN,0240000004
3,1.0,1.0,ISIDRO FABELA SUR S/N,5245887,NaN,NaN,NaN,3813.0,NaN,NaN,NaN,NaN,2014-11-03 11:27:32.0,0240100101,0240100101000000,H. AYUNTAMIENTO,DE ATLACOMULCO,(PLAZA MIGUEL HIDALGO),NaN,50450.0,14.0,15.0,AV. ISIDRO FABELA SUR S/N,1.0,528.0,NaN,MAT,NaN,NaN,0240100101
4,2.0,1.0,"AVENIDA FABELA ISIDRO NTE,# S/N,COL.CENT S/N S/N",4092285,NaN,3341520,NaN,5493.0,NaN,1560.0,NaN,0240100201000000,2013-09-25 10:06:21.0,0240100201,0240100201000000,IGLESIA CATOLICA,NaN,NaN,NaN,50450.0,14.0,15.0,"AVENIDA FABELA ISIDRO NTE,# S/N,COL.CENTRO,CP....",1.0,528.0,NaN,NaN,NaN,NaN,0240100201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29753,308.0,15.0,SIN NOMBRE S/N,41625,NaN,NaN,NaN,375.0,NaN,NaN,NaN,0241530801000000,2015-03-12 10:36:06.0,0241530821,0241530821000000,MARTHA LUCILA,SERVIN,NUÑEZ,SENM830814KS2,50475.0,14.0,15.0,SIN NOMBRE S/N,1.0,555.0,tathy311@gmail.com,SENM,NaN,NaN,0241530821
29754,308.0,15.0,SIN NOMBRE S/N S/N,12987,NaN,NaN,NaN,117.0,NaN,NaN,NaN,0241530807000000,2015-12-01 09:17:27.0,0241530822,0241530822000000,PEDRO,ORDOÑEZ,MENDOZA,NaN,50475.0,14.0,15.0,SIN NOMBRE S/N S/N,1.0,555.0,NaN,NaN,NaN,NaN,0241530822
29755,308.0,15.0,SIN NOMBRE S/N S/N,89910,NaN,NaN,NaN,810.0,NaN,NaN,NaN,0241530807000000,2016-03-04 15:23:03.0,0241530823,0241530823000000,MIRIAM,MELCHOR,DE LA CRUZ,NaN,50475.0,14.0,15.0,SIN NOMBRE S/N S/N,1.0,555.0,NaN,NaN,NaN,NaN,0241530823
29756,308.0,15.0,SIN NOMBRE S/N S/N,568004,NaN,NaN,NaN,9986.0,NaN,NaN,NaN,0241580410000000,2016-03-23 11:04:24.0,0241530824,0241530824000000,FELIPE,HIPOLITO,RUIZ,NaN,50475.0,14.0,15.0,SIN NOMBRE S/N S/N,1.0,554.0,NaN,NaN,NaN,NaN,0241530824


In [70]:
#Al cruzar se generara x y y al existir la columna simultaneamente en las bases, se dejará la que tenga mayor peso
predial_general['CLAVE_CATASTRAL']=predial_general.CLAVE_CATASTRAL_x.combine_first(predial_general.CLAVE_CATASTRAL_y)
#ya que se tiene la columna correcta, se eliminan las generadas como x y Y
columns_to_drop = ['CLAVE_CATASTRAL_x', 'CLAVE_CATASTRAL_y']
predial_general = predial_general.drop(columns_to_drop, axis=1)
predial_general


,CLAVEMANZANA,CLAVEZONA,DIRECCION,VALORTERRENO,VALORTERRENOCOMUN,VALORCONSTPROP,VALORCONSTCOMUN,SUPERFICIE,SUPERFICIETERRCOMUN,SUPERFICIECONST,SUPERFICIECONSTCOMUN,CLAVEANTERIOR,FECHAALTA,CLAVECATASTRAL,NOMBRE,APATERNO,AMATERNO,RFC,CLAVECP,CLAVEMUNICIPIO,CLAVEENTIDAD,CALLE,CLAVESTATUS,CLAVEASENTAMIENTO,EMAIL,CURP,ASENTAMIENTO_NR,CLAVECP_NR,CLAVE_CATASTRAL
0,NaN,NaN,PRUEBA 1 1,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,2013-09-25 12:59:34.0,0240000000PRUEBA,PRUEBA 1,NaN,NaN,NaN,51260.0,7.0,15.0,PRUEBA 1,1.0,1054.0,NaN,NaN,NaN,NaN,0240000000
1,NaN,NaN,PRUEBA 2 2,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,NaN,2013-09-25 13:33:11.0,0240000001PRUEBA,PRUEBA 2,NaN,NaN,NaN,56766.0,17.0,15.0,PRUEBA 2 2,1.0,9282.0,NaN,NaN,NaN,NaN,0240000001
2,NaN,NaN,PRUEBA 111 111,300000,NaN,145000,NaN,100.0,NaN,200.0,NaN,NaN,2013-10-10 10:47:42.0,0240000004PRUEBA,PRUEBA 1111,PRUEBA,PRUEBA,NaN,52542.0,6.0,15.0,PRUEBA 111 111,1.0,6881.0,NaN,NaN,NaN,NaN,0240000004
3,1.0,1.0,ISIDRO FABELA SUR S/N,5245887,NaN,NaN,NaN,3813.0,NaN,NaN,NaN,NaN,2014-11-03 11:27:32.0,0240100101000000,H. AYUNTAMIENTO,DE ATLACOMULCO,(PLAZA MIGUEL HIDALGO),NaN,50450.0,14.0,15.0,AV. ISIDRO FABELA SUR S/N,1.0,528.0,NaN,MAT,NaN,NaN,0240100101
4,2.0,1.0,"AVENIDA FABELA ISIDRO NTE,# S/N,COL.CENT S/N S/N",4092285,NaN,3341520,NaN,5493.0,NaN,1560.0,NaN,0240100201000000,2013-09-25 10:06:21.0,0240100201000000,IGLESIA CATOLICA,NaN,NaN,NaN,50450.0,14.0,15.0,"AVENIDA FABELA ISIDRO NTE,# S/N,COL.CENTRO,CP....",1.0,528.0,NaN,NaN,NaN,NaN,0240100201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29753,308.0,15.0,SIN NOMBRE S/N,41625,NaN,NaN,NaN,375.0,NaN,NaN,NaN,0241530801000000,2015-03-12 10:36:06.0,0241530821000000,MARTHA LUCILA,SERVIN,NUÑEZ,SENM830814KS2,50475.0,14.0,15.0,SIN NOMBRE S/N,1.0,555.0,tathy311@gmail.com,SENM,NaN,NaN,0241530821
29754,308.0,15.0,SIN NOMBRE S/N S/N,12987,NaN,NaN,NaN,117.0,NaN,NaN,NaN,0241530807000000,2015-12-01 09:17:27.0,0241530822000000,PEDRO,ORDOÑEZ,MENDOZA,NaN,50475.0,14.0,15.0,SIN NOMBRE S/N S/N,1.0,555.0,NaN,NaN,NaN,NaN,0241530822
29755,308.0,15.0,SIN NOMBRE S/N S/N,89910,NaN,NaN,NaN,810.0,NaN,NaN,NaN,0241530807000000,2016-03-04 15:23:03.0,0241530823000000,MIRIAM,MELCHOR,DE LA CRUZ,NaN,50475.0,14.0,15.0,SIN NOMBRE S/N S/N,1.0,555.0,NaN,NaN,NaN,NaN,0241530823
29756,308.0,15.0,SIN NOMBRE S/N S/N,568004,NaN,NaN,NaN,9986.0,NaN,NaN,NaN,0241580410000000,2016-03-23 11:04:24.0,0241530824000000,FELIPE,HIPOLITO,RUIZ,NaN,50475.0,14.0,15.0,SIN NOMBRE S/N S/N,1.0,554.0,NaN,NaN,NaN,NaN,0241530824


In [71]:
#Revisar q no haya vacíos
predial_general[predial_general.CLAVE_CATASTRAL.isna()]
predial_general[predial_general.CLAVECATASTRAL.isna()]

,CLAVEMANZANA,CLAVEZONA,DIRECCION,VALORTERRENO,VALORTERRENOCOMUN,VALORCONSTPROP,VALORCONSTCOMUN,SUPERFICIE,SUPERFICIETERRCOMUN,SUPERFICIECONST,SUPERFICIECONSTCOMUN,CLAVEANTERIOR,FECHAALTA,CLAVECATASTRAL,NOMBRE,APATERNO,AMATERNO,RFC,CLAVECP,CLAVEMUNICIPIO,CLAVEENTIDAD,CALLE,CLAVESTATUS,CLAVEASENTAMIENTO,EMAIL,CURP,ASENTAMIENTO_NR,CLAVECP_NR,CLAVE_CATASTRAL


In [72]:
len(predial_general.CLAVECATASTRAL.unique())

29758

In [73]:
#revisión de columnas entre predial general (cruce 1) y predial adeudos
predial_general.columns[predial_general.columns.isin(pred_deu.columns)]
falta_deu= predial_general.columns[~predial_general.columns.isin(pred_deu.columns)]
falta_deu

Index(['CLAVEMANZANA', 'CLAVEZONA', 'DIRECCION', 'VALORTERRENO',
       'VALORTERRENOCOMUN', 'VALORCONSTPROP', 'VALORCONSTCOMUN', 'SUPERFICIE',
       'SUPERFICIETERRCOMUN', 'SUPERFICIECONST', 'SUPERFICIECONSTCOMUN',
       'CLAVEANTERIOR', 'FECHAALTA', 'NOMBRE', 'APATERNO', 'AMATERNO', 'RFC',
       'CLAVECP', 'CLAVEMUNICIPIO', 'CLAVEENTIDAD', 'CALLE', 'CLAVESTATUS',
       'CLAVEASENTAMIENTO', 'EMAIL', 'CURP', 'ASENTAMIENTO_NR', 'CLAVECP_NR'],
      dtype='object')

Cruce 5: Predial general vs Predial - Deudores

In [74]:
cruce_pred1=pd.merge(predial_general[['CLAVECATASTRAL','CLAVE_CATASTRAL','CLAVEMANZANA', 'CLAVEZONA', 'DIRECCION', 'VALORTERRENO',
       'VALORTERRENOCOMUN', 'VALORCONSTPROP', 'VALORCONSTCOMUN', 'SUPERFICIE',
       'SUPERFICIETERRCOMUN', 'SUPERFICIECONST', 'SUPERFICIECONSTCOMUN',
       'CLAVEANTERIOR', 'FECHAALTA', 'NOMBRE', 'APATERNO', 'AMATERNO', 'RFC',
       'CLAVECP', 'CLAVEMUNICIPIO', 'CLAVEENTIDAD', 'CALLE', 'CLAVESTATUS',
       'CLAVEASENTAMIENTO', 'EMAIL', 'CURP', 'ASENTAMIENTO_NR', 'CLAVECP_NR']],pred_deu, on="CLAVECATASTRAL",  how='outer', indicator=True)
cruce_pred1

,CLAVECATASTRAL,CLAVE_CATASTRAL_x,CLAVEMANZANA,CLAVEZONA,DIRECCION,VALORTERRENO,VALORTERRENOCOMUN,VALORCONSTPROP,VALORCONSTCOMUN,SUPERFICIE,SUPERFICIETERRCOMUN,SUPERFICIECONST,SUPERFICIECONSTCOMUN,CLAVEANTERIOR,FECHAALTA,NOMBRE,APATERNO,AMATERNO,RFC,CLAVECP,CLAVEMUNICIPIO,CLAVEENTIDAD,CALLE,CLAVESTATUS,CLAVEASENTAMIENTO,EMAIL,CURP,ASENTAMIENTO_NR,CLAVECP_NR,EJERCICIO,CLAVEPERIODOINI,CLAVEPERIODOFIN,VALORCATASTRAL,STATUS,CLAVE_CATASTRAL_y,_merge
0,0240000000PRUEBA,0240000000,NaN,NaN,PRUEBA 1 1,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,2013-09-25 12:59:34.0,PRUEBA 1,NaN,NaN,NaN,51260.0,7.0,15.0,PRUEBA 1,1.0,1054.0,NaN,NaN,NaN,NaN,2018.0,1.0,12.0,245321.0,Adeudo,0240000000,both
1,0240000001PRUEBA,0240000001,NaN,NaN,PRUEBA 2 2,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,NaN,2013-09-25 13:33:11.0,PRUEBA 2,NaN,NaN,NaN,56766.0,17.0,15.0,PRUEBA 2 2,1.0,9282.0,NaN,NaN,NaN,NaN,2012.0,1.0,12.0,22222.0,Adeudo,0240000001,both
2,0240000004PRUEBA,0240000004,NaN,NaN,PRUEBA 111 111,300000,NaN,145000,NaN,100.0,NaN,200.0,NaN,NaN,2013-10-10 10:47:42.0,PRUEBA 1111,PRUEBA,PRUEBA,NaN,52542.0,6.0,15.0,PRUEBA 111 111,1.0,6881.0,NaN,NaN,NaN,NaN,2011.0,1.0,12.0,400000.0,Adeudo,0240000004,both
3,0240100101000000,0240100101,1.0,1.0,ISIDRO FABELA SUR S/N,5245887,NaN,NaN,NaN,3813.0,NaN,NaN,NaN,NaN,2014-11-03 11:27:32.0,H. AYUNTAMIENTO,DE ATLACOMULCO,(PLAZA MIGUEL HIDALGO),NaN,50450.0,14.0,15.0,AV. ISIDRO FABELA SUR S/N,1.0,528.0,NaN,MAT,NaN,NaN,2015.0,1.0,12.0,4348116.0,Adeudo,0240100101,both
4,0240100201000000,0240100201,2.0,1.0,"AVENIDA FABELA ISIDRO NTE,# S/N,COL.CENT S/N S/N",4092285,NaN,3341520,NaN,5493.0,NaN,1560.0,NaN,0240100201000000,2013-09-25 10:06:21.0,IGLESIA CATOLICA,NaN,NaN,NaN,50450.0,14.0,15.0,"AVENIDA FABELA ISIDRO NTE,# S/N,COL.CENTRO,CP....",1.0,528.0,NaN,NaN,NaN,NaN,2008.0,1.0,12.0,4697952.0,Pago Municipal,0240100201,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29753,0241530821000000,0241530821,308.0,15.0,SIN NOMBRE S/N,41625,NaN,NaN,NaN,375.0,NaN,NaN,NaN,0241530801000000,2015-03-12 10:36:06.0,MARTHA LUCILA,SERVIN,NUÑEZ,SENM830814KS2,50475.0,14.0,15.0,SIN NOMBRE S/N,1.0,555.0,tathy311@gmail.com,SENM,NaN,NaN,2016.0,1.0,12.0,38250.0,Pagada,0241530821,both
29754,0241530822000000,0241530822,308.0,15.0,SIN NOMBRE S/N S/N,12987,NaN,NaN,NaN,117.0,NaN,NaN,NaN,0241530807000000,2015-12-01 09:17:27.0,PEDRO,ORDOÑEZ,MENDOZA,NaN,50475.0,14.0,15.0,SIN NOMBRE S/N S/N,1.0,555.0,NaN,NaN,NaN,NaN,2017.0,1.0,12.0,12987.0,Pago Municipal,0241530822,both
29755,0241530823000000,0241530823,308.0,15.0,SIN NOMBRE S/N S/N,89910,NaN,NaN,NaN,810.0,NaN,NaN,NaN,0241530807000000,2016-03-04 15:23:03.0,MIRIAM,MELCHOR,DE LA CRUZ,NaN,50475.0,14.0,15.0,SIN NOMBRE S/N S/N,1.0,555.0,NaN,NaN,NaN,NaN,2017.0,1.0,12.0,89910.0,Pago Municipal,0241530823,both
29756,0241530824000000,0241530824,308.0,15.0,SIN NOMBRE S/N S/N,568004,NaN,NaN,NaN,9986.0,NaN,NaN,NaN,0241580410000000,2016-03-23 11:04:24.0,FELIPE,HIPOLITO,RUIZ,NaN,50475.0,14.0,15.0,SIN NOMBRE S/N S/N,1.0,554.0,NaN,NaN,NaN,NaN,2011.0,1.0,12.0,344019.0,Pago Municipal,0241530824,both


In [75]:
#Al cruzar se generara x y y al existir la columna simultaneamente en las bases, se dejará la que tenga mayor peso
cruce_pred1['CLAVE_CATASTRAL']=cruce_pred1.CLAVE_CATASTRAL_x.combine_first(cruce_pred1.CLAVE_CATASTRAL_y)
#ya que se tiene la columna correcta, se eliminan las generadas como x y Y
columns_to_drop = ['CLAVE_CATASTRAL_x', 'CLAVE_CATASTRAL_y']
cruce_pred1 = cruce_pred1.drop(columns_to_drop, axis=1)
cruce_pred1

,CLAVECATASTRAL,CLAVEMANZANA,CLAVEZONA,DIRECCION,VALORTERRENO,VALORTERRENOCOMUN,VALORCONSTPROP,VALORCONSTCOMUN,SUPERFICIE,SUPERFICIETERRCOMUN,SUPERFICIECONST,SUPERFICIECONSTCOMUN,CLAVEANTERIOR,FECHAALTA,NOMBRE,APATERNO,AMATERNO,RFC,CLAVECP,CLAVEMUNICIPIO,CLAVEENTIDAD,CALLE,CLAVESTATUS,CLAVEASENTAMIENTO,EMAIL,CURP,ASENTAMIENTO_NR,CLAVECP_NR,EJERCICIO,CLAVEPERIODOINI,CLAVEPERIODOFIN,VALORCATASTRAL,STATUS,_merge,CLAVE_CATASTRAL
0,0240000000PRUEBA,NaN,NaN,PRUEBA 1 1,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,2013-09-25 12:59:34.0,PRUEBA 1,NaN,NaN,NaN,51260.0,7.0,15.0,PRUEBA 1,1.0,1054.0,NaN,NaN,NaN,NaN,2018.0,1.0,12.0,245321.0,Adeudo,both,0240000000
1,0240000001PRUEBA,NaN,NaN,PRUEBA 2 2,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,NaN,2013-09-25 13:33:11.0,PRUEBA 2,NaN,NaN,NaN,56766.0,17.0,15.0,PRUEBA 2 2,1.0,9282.0,NaN,NaN,NaN,NaN,2012.0,1.0,12.0,22222.0,Adeudo,both,0240000001
2,0240000004PRUEBA,NaN,NaN,PRUEBA 111 111,300000,NaN,145000,NaN,100.0,NaN,200.0,NaN,NaN,2013-10-10 10:47:42.0,PRUEBA 1111,PRUEBA,PRUEBA,NaN,52542.0,6.0,15.0,PRUEBA 111 111,1.0,6881.0,NaN,NaN,NaN,NaN,2011.0,1.0,12.0,400000.0,Adeudo,both,0240000004
3,0240100101000000,1.0,1.0,ISIDRO FABELA SUR S/N,5245887,NaN,NaN,NaN,3813.0,NaN,NaN,NaN,NaN,2014-11-03 11:27:32.0,H. AYUNTAMIENTO,DE ATLACOMULCO,(PLAZA MIGUEL HIDALGO),NaN,50450.0,14.0,15.0,AV. ISIDRO FABELA SUR S/N,1.0,528.0,NaN,MAT,NaN,NaN,2015.0,1.0,12.0,4348116.0,Adeudo,both,0240100101
4,0240100201000000,2.0,1.0,"AVENIDA FABELA ISIDRO NTE,# S/N,COL.CENT S/N S/N",4092285,NaN,3341520,NaN,5493.0,NaN,1560.0,NaN,0240100201000000,2013-09-25 10:06:21.0,IGLESIA CATOLICA,NaN,NaN,NaN,50450.0,14.0,15.0,"AVENIDA FABELA ISIDRO NTE,# S/N,COL.CENTRO,CP....",1.0,528.0,NaN,NaN,NaN,NaN,2008.0,1.0,12.0,4697952.0,Pago Municipal,both,0240100201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29753,0241530821000000,308.0,15.0,SIN NOMBRE S/N,41625,NaN,NaN,NaN,375.0,NaN,NaN,NaN,0241530801000000,2015-03-12 10:36:06.0,MARTHA LUCILA,SERVIN,NUÑEZ,SENM830814KS2,50475.0,14.0,15.0,SIN NOMBRE S/N,1.0,555.0,tathy311@gmail.com,SENM,NaN,NaN,2016.0,1.0,12.0,38250.0,Pagada,both,0241530821
29754,0241530822000000,308.0,15.0,SIN NOMBRE S/N S/N,12987,NaN,NaN,NaN,117.0,NaN,NaN,NaN,0241530807000000,2015-12-01 09:17:27.0,PEDRO,ORDOÑEZ,MENDOZA,NaN,50475.0,14.0,15.0,SIN NOMBRE S/N S/N,1.0,555.0,NaN,NaN,NaN,NaN,2017.0,1.0,12.0,12987.0,Pago Municipal,both,0241530822
29755,0241530823000000,308.0,15.0,SIN NOMBRE S/N S/N,89910,NaN,NaN,NaN,810.0,NaN,NaN,NaN,0241530807000000,2016-03-04 15:23:03.0,MIRIAM,MELCHOR,DE LA CRUZ,NaN,50475.0,14.0,15.0,SIN NOMBRE S/N S/N,1.0,555.0,NaN,NaN,NaN,NaN,2017.0,1.0,12.0,89910.0,Pago Municipal,both,0241530823
29756,0241530824000000,308.0,15.0,SIN NOMBRE S/N S/N,568004,NaN,NaN,NaN,9986.0,NaN,NaN,NaN,0241580410000000,2016-03-23 11:04:24.0,FELIPE,HIPOLITO,RUIZ,NaN,50475.0,14.0,15.0,SIN NOMBRE S/N S/N,1.0,554.0,NaN,NaN,NaN,NaN,2011.0,1.0,12.0,344019.0,Pago Municipal,both,0241530824


In [76]:
cruce_pred1[cruce_pred1.CLAVECATASTRAL.isna()]

,CLAVECATASTRAL,CLAVEMANZANA,CLAVEZONA,DIRECCION,VALORTERRENO,VALORTERRENOCOMUN,VALORCONSTPROP,VALORCONSTCOMUN,SUPERFICIE,SUPERFICIETERRCOMUN,SUPERFICIECONST,SUPERFICIECONSTCOMUN,CLAVEANTERIOR,FECHAALTA,NOMBRE,APATERNO,AMATERNO,RFC,CLAVECP,CLAVEMUNICIPIO,CLAVEENTIDAD,CALLE,CLAVESTATUS,CLAVEASENTAMIENTO,EMAIL,CURP,ASENTAMIENTO_NR,CLAVECP_NR,EJERCICIO,CLAVEPERIODOINI,CLAVEPERIODOFIN,VALORCATASTRAL,STATUS,_merge,CLAVE_CATASTRAL


In [77]:
len(cruce_pred1.CLAVECATASTRAL.unique())

29758

In [114]:
#guardar bases
cruce_pred1.to_csv(r'I:\Mi unidad\Cruces_predial_otros\Ixtapan/Ixtapan_predial_completo.csv',encoding="utf-8-sig")

In [79]:
#eliminar la columna de merge en la base a trabajar de cruce_pred1
cruce_pred1=cruce_pred1.drop(['_merge'], axis=1)

In [80]:
cruce_pred1.columns[cruce_pred1.columns.isin(cruce_predial_completo.columns)]
falta_cruce_pred1= cruce_pred1.columns[~cruce_pred1.columns.isin(cruce_predial_completo.columns)]
falta_cruce_pred1

Index(['CLAVEMANZANA', 'CLAVEZONA', 'DIRECCION', 'VALORTERRENO',
       'VALORTERRENOCOMUN', 'VALORCONSTPROP', 'VALORCONSTCOMUN', 'SUPERFICIE',
       'SUPERFICIETERRCOMUN', 'SUPERFICIECONST', 'SUPERFICIECONSTCOMUN',
       'CLAVEANTERIOR', 'FECHAALTA', 'NOMBRE', 'APATERNO', 'AMATERNO', 'RFC',
       'CLAVECP', 'CLAVEMUNICIPIO', 'CLAVEENTIDAD', 'CALLE', 'CLAVESTATUS',
       'CLAVEASENTAMIENTO', 'EMAIL', 'CURP', 'ASENTAMIENTO_NR', 'CLAVECP_NR',
       'EJERCICIO', 'CLAVEPERIODOINI', 'CLAVEPERIODOFIN', 'VALORCATASTRAL',
       'STATUS'],
      dtype='object')

Cruce 6: Cruce de Predial general + Predial - Deudores vs Predial_OneDrive + CURT

In [81]:
cruce_CURT_predial = pd.merge(cruce_pred1[['CLAVE_CATASTRAL', 'CLAVECATASTRAL', 'CLAVEMANZANA', 'CLAVEZONA', 'DIRECCION', 'VALORTERRENO',
       'VALORTERRENOCOMUN', 'VALORCONSTPROP', 'VALORCONSTCOMUN', 'SUPERFICIE',
       'SUPERFICIETERRCOMUN', 'SUPERFICIECONST', 'SUPERFICIECONSTCOMUN',
       'CLAVEANTERIOR', 'FECHAALTA', 'NOMBRE', 'APATERNO', 'AMATERNO', 'RFC',
       'CLAVECP', 'CLAVEMUNICIPIO', 'CLAVEENTIDAD', 'CALLE', 'CLAVESTATUS',
       'CLAVEASENTAMIENTO', 'EMAIL', 'CURP', 'ASENTAMIENTO_NR', 'CLAVECP_NR',
       'EJERCICIO', 'CLAVEPERIODOINI', 'CLAVEPERIODOFIN', 'VALORCATASTRAL',
       'STATUS']], cruce_predial_completo, on='CLAVE_CATASTRAL', how = 'outer', indicator=True)
cruce_CURT_predial

,CLAVE_CATASTRAL,CLAVECATASTRAL_x,CLAVEMANZANA,CLAVEZONA,DIRECCION,VALORTERRENO,VALORTERRENOCOMUN,VALORCONSTPROP,VALORCONSTCOMUN,SUPERFICIE,SUPERFICIETERRCOMUN,SUPERFICIECONST,SUPERFICIECONSTCOMUN,CLAVEANTERIOR,FECHAALTA,NOMBRE,APATERNO,AMATERNO,RFC,CLAVECP,CLAVEMUNICIPIO,CLAVEENTIDAD,CALLE,CLAVESTATUS,CLAVEASENTAMIENTO,EMAIL,CURP,ASENTAMIENTO_NR,CLAVECP_NR,EJERCICIO,CLAVEPERIODOINI,CLAVEPERIODOFIN,VALORCATASTRAL,STATUS,fol_rec,nombre,rfc,ubicacion,domicilio_fiscal,sup_terreno,sup_construc,valor_catastral,periodo,periodo_anual,fec_pago,importe,recargos,multas,descuento,fol_fisc_uuid,facturado,CLAVE_MANZ,CLAVE_PREDIO,CLAVECATASTRAL_y,cve_ent,cve_mun,cve_loc,curt,notas,geometry,mun,zona,manz,_merge
0,0240000000,0240000000PRUEBA,NaN,NaN,PRUEBA 1 1,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,2013-09-25 12:59:34.0,PRUEBA 1,NaN,NaN,NaN,51260.0,7.0,15.0,PRUEBA 1,1.0,1054.0,NaN,NaN,NaN,NaN,2018.0,1.0,12.0,245321.0,Adeudo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,left_only
1,0240000001,0240000001PRUEBA,NaN,NaN,PRUEBA 2 2,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,NaN,2013-09-25 13:33:11.0,PRUEBA 2,NaN,NaN,NaN,56766.0,17.0,15.0,PRUEBA 2 2,1.0,9282.0,NaN,NaN,NaN,NaN,2012.0,1.0,12.0,22222.0,Adeudo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,left_only
2,0240000004,0240000004PRUEBA,NaN,NaN,PRUEBA 111 111,300000,NaN,145000,NaN,100.0,NaN,200.0,NaN,NaN,2013-10-10 10:47:42.0,PRUEBA 1111,PRUEBA,PRUEBA,NaN,52542.0,6.0,15.0,PRUEBA 111 111,1.0,6881.0,NaN,NaN,NaN,NaN,2011.0,1.0,12.0,400000.0,Adeudo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,left_only
3,0240100101,0240100101000000,1.0,1.0,ISIDRO FABELA SUR S/N,5245887,NaN,NaN,NaN,3813.0,NaN,NaN,NaN,NaN,2014-11-03 11:27:32.0,H. AYUNTAMIENTO,DE ATLACOMULCO,(PLAZA MIGUEL HIDALGO),NaN,50450.0,14.0,15.0,AV. ISIDRO FABELA SUR S/N,1.0,528.0,NaN,MAT,NaN,NaN,2015.0,1.0,12.0,4348116.0,Adeudo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,left_only
4,0240100201,0240100201000000,2.0,1.0,"AVENIDA FABELA ISIDRO NTE,# S/N,COL.CENT S/N S/N",4092285,NaN,3341520,NaN,5493.0,NaN,1560.0,NaN,0240100201000000,2013-09-25 10:06:21.0,IGLESIA CATOLICA,NaN,NaN,NaN,50450.0,14.0,15.0,"AVENIDA FABELA ISIDRO NTE,# S/N,COL.CENTRO,CP....",1.0,528.0,NaN,NaN,NaN,NaN,2008.0,1.0,12.0,4697952.0,Pago Municipal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151019,0240404702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TM045172,PEREZ UGALDE PABLO,XAXX010101000,"VENUSTIANO CARRANZA, #SN, CUENDO, CP 50486, AT...","NICOLAS BRAVO, #SN, LA LOMA, CP 50355, ACAMBAY...",237.0,NaN,47400.0,DE 01-2019 AL 12-2019,2019,2022-09-07,214.0,107.0,NaN,75.0,45420B3B-691A-4DC5-AC4D-B6C0B3DD41AA,NaN,0240404700,0240404702,0240404702000000,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-99.94668 19.79828, -99.94506 19.797...",024,04,047,right_only
151020,0240326401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TM051561,MARIA MENDOZA VDA. DE SERRANO Y JUAN SERRANO M...,XAXX010101000,CALLE SIN NOMBRE SAN LORENZO TLACOTEPEC,NaN,4934.0,NaN,1393273.0,DE 01-2017 AL 12-2017,2017,2022-11-23,3294.0,1644.0,NaN,1842.0,058DCB3A-CE41-451B-B86A-7D38CA6D638F,NaN,0240326400,0240326401,0240326401000000,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-99.91476 19.81252, -99.91570 19.811...",024,03,264,right_only
151021,0240405203,NaN,NaN,NaN,N

In [82]:
#Al cruzar se generara x y y al existir la columna simultaneamente en las bases, se dejará la que tenga mayor peso
cruce_CURT_predial['CLAVECATASTRAL']=cruce_CURT_predial.CLAVECATASTRAL_x.combine_first(cruce_CURT_predial.CLAVECATASTRAL_y)
#ya que se tiene la columna correcta, se eliminan las generadas como x y Y
columns_to_drop = ['CLAVECATASTRAL_x', 'CLAVECATASTRAL_y']
cruce_CURT_predial = cruce_CURT_predial.drop(columns_to_drop, axis=1)
cruce_CURT_predial

,CLAVE_CATASTRAL,CLAVEMANZANA,CLAVEZONA,DIRECCION,VALORTERRENO,VALORTERRENOCOMUN,VALORCONSTPROP,VALORCONSTCOMUN,SUPERFICIE,SUPERFICIETERRCOMUN,SUPERFICIECONST,SUPERFICIECONSTCOMUN,CLAVEANTERIOR,FECHAALTA,NOMBRE,APATERNO,AMATERNO,RFC,CLAVECP,CLAVEMUNICIPIO,CLAVEENTIDAD,CALLE,CLAVESTATUS,CLAVEASENTAMIENTO,EMAIL,CURP,ASENTAMIENTO_NR,CLAVECP_NR,EJERCICIO,CLAVEPERIODOINI,CLAVEPERIODOFIN,VALORCATASTRAL,STATUS,fol_rec,nombre,rfc,ubicacion,domicilio_fiscal,sup_terreno,sup_construc,valor_catastral,periodo,periodo_anual,fec_pago,importe,recargos,multas,descuento,fol_fisc_uuid,facturado,CLAVE_MANZ,CLAVE_PREDIO,cve_ent,cve_mun,cve_loc,curt,notas,geometry,mun,zona,manz,_merge,CLAVECATASTRAL
0,0240000000,NaN,NaN,PRUEBA 1 1,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,2013-09-25 12:59:34.0,PRUEBA 1,NaN,NaN,NaN,51260.0,7.0,15.0,PRUEBA 1,1.0,1054.0,NaN,NaN,NaN,NaN,2018.0,1.0,12.0,245321.0,Adeudo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,left_only,0240000000PRUEBA
1,0240000001,NaN,NaN,PRUEBA 2 2,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,NaN,2013-09-25 13:33:11.0,PRUEBA 2,NaN,NaN,NaN,56766.0,17.0,15.0,PRUEBA 2 2,1.0,9282.0,NaN,NaN,NaN,NaN,2012.0,1.0,12.0,22222.0,Adeudo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,left_only,0240000001PRUEBA
2,0240000004,NaN,NaN,PRUEBA 111 111,300000,NaN,145000,NaN,100.0,NaN,200.0,NaN,NaN,2013-10-10 10:47:42.0,PRUEBA 1111,PRUEBA,PRUEBA,NaN,52542.0,6.0,15.0,PRUEBA 111 111,1.0,6881.0,NaN,NaN,NaN,NaN,2011.0,1.0,12.0,400000.0,Adeudo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,left_only,0240000004PRUEBA
3,0240100101,1.0,1.0,ISIDRO FABELA SUR S/N,5245887,NaN,NaN,NaN,3813.0,NaN,NaN,NaN,NaN,2014-11-03 11:27:32.0,H. AYUNTAMIENTO,DE ATLACOMULCO,(PLAZA MIGUEL HIDALGO),NaN,50450.0,14.0,15.0,AV. ISIDRO FABELA SUR S/N,1.0,528.0,NaN,MAT,NaN,NaN,2015.0,1.0,12.0,4348116.0,Adeudo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,left_only,0240100101000000
4,0240100201,2.0,1.0,"AVENIDA FABELA ISIDRO NTE,# S/N,COL.CENT S/N S/N",4092285,NaN,3341520,NaN,5493.0,NaN,1560.0,NaN,0240100201000000,2013-09-25 10:06:21.0,IGLESIA CATOLICA,NaN,NaN,NaN,50450.0,14.0,15.0,"AVENIDA FABELA ISIDRO NTE,# S/N,COL.CENTRO,CP....",1.0,528.0,NaN,NaN,NaN,NaN,2008.0,1.0,12.0,4697952.0,Pago Municipal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,left_only,0240100201000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151019,0240404702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TM045172,PEREZ UGALDE PABLO,XAXX010101000,"VENUSTIANO CARRANZA, #SN, CUENDO, CP 50486, AT...","NICOLAS BRAVO, #SN, LA LOMA, CP 50355, ACAMBAY...",237.0,NaN,47400.0,DE 01-2019 AL 12-2019,2019,2022-09-07,214.0,107.0,NaN,75.0,45420B3B-691A-4DC5-AC4D-B6C0B3DD41AA,NaN,0240404700,0240404702,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-99.94668 19.79828, -99.94506 19.797...",024,04,047,right_only,0240404702000000
151020,0240326401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TM051561,MARIA MENDOZA VDA. DE SERRANO Y JUAN SERRANO M...,XAXX010101000,CALLE SIN NOMBRE SAN LORENZO TLACOTEPEC,NaN,4934.0,NaN,1393273.0,DE 01-2017 AL 12-2017,2017,2022-11-23,3294.0,1644.0,NaN,1842.0,058DCB3A-CE41-451B-B86A-7D38CA6D638F,NaN,0240326400,0240326401,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-99.91476 19.81252, -99.91570 19.811...",024,03,264,right_only,0240326401000000
151021,0240405203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [83]:
cruce_CURT_predial= cruce_CURT_predial.rename(columns = {'_merge' : 'ID_predial'})

In [84]:
faltan = base_predial[~base_predial.CLAVECATASTRAL.isin(cruce_CURT_predial.CLAVECATASTRAL)]
faltan.shape

(3, 21)

In [85]:
#Se concatenan los registros de base predial que no cruzaron
base_final_CURT_Predial = pd.concat([cruce_CURT_predial,faltan])
base_final_CURT_Predial

,CLAVE_CATASTRAL,CLAVEMANZANA,CLAVEZONA,DIRECCION,VALORTERRENO,VALORTERRENOCOMUN,VALORCONSTPROP,VALORCONSTCOMUN,SUPERFICIE,SUPERFICIETERRCOMUN,SUPERFICIECONST,SUPERFICIECONSTCOMUN,CLAVEANTERIOR,FECHAALTA,NOMBRE,APATERNO,AMATERNO,RFC,CLAVECP,CLAVEMUNICIPIO,CLAVEENTIDAD,CALLE,CLAVESTATUS,CLAVEASENTAMIENTO,EMAIL,CURP,ASENTAMIENTO_NR,CLAVECP_NR,EJERCICIO,CLAVEPERIODOINI,CLAVEPERIODOFIN,VALORCATASTRAL,STATUS,fol_rec,nombre,rfc,ubicacion,domicilio_fiscal,sup_terreno,sup_construc,valor_catastral,periodo,periodo_anual,fec_pago,importe,recargos,multas,descuento,fol_fisc_uuid,facturado,CLAVE_MANZ,CLAVE_PREDIO,cve_ent,cve_mun,cve_loc,curt,notas,geometry,mun,zona,manz,ID_predial,CLAVECATASTRAL
0,0240000000,NaN,NaN,PRUEBA 1 1,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,2013-09-25 12:59:34.0,PRUEBA 1,NaN,NaN,NaN,51260.0,7.0,15.0,PRUEBA 1,1.0,1054.0,NaN,NaN,NaN,NaN,2018.0,1.0,12.0,245321.0,Adeudo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,left_only,0240000000PRUEBA
1,0240000001,NaN,NaN,PRUEBA 2 2,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,NaN,2013-09-25 13:33:11.0,PRUEBA 2,NaN,NaN,NaN,56766.0,17.0,15.0,PRUEBA 2 2,1.0,9282.0,NaN,NaN,NaN,NaN,2012.0,1.0,12.0,22222.0,Adeudo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,left_only,0240000001PRUEBA
2,0240000004,NaN,NaN,PRUEBA 111 111,300000,NaN,145000,NaN,100.0,NaN,200.0,NaN,NaN,2013-10-10 10:47:42.0,PRUEBA 1111,PRUEBA,PRUEBA,NaN,52542.0,6.0,15.0,PRUEBA 111 111,1.0,6881.0,NaN,NaN,NaN,NaN,2011.0,1.0,12.0,400000.0,Adeudo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,left_only,0240000004PRUEBA
3,0240100101,1.0,1.0,ISIDRO FABELA SUR S/N,5245887,NaN,NaN,NaN,3813.0,NaN,NaN,NaN,NaN,2014-11-03 11:27:32.0,H. AYUNTAMIENTO,DE ATLACOMULCO,(PLAZA MIGUEL HIDALGO),NaN,50450.0,14.0,15.0,AV. ISIDRO FABELA SUR S/N,1.0,528.0,NaN,MAT,NaN,NaN,2015.0,1.0,12.0,4348116.0,Adeudo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,left_only,0240100101000000
4,0240100201,2.0,1.0,"AVENIDA FABELA ISIDRO NTE,# S/N,COL.CENT S/N S/N",4092285,NaN,3341520,NaN,5493.0,NaN,1560.0,NaN,0240100201000000,2013-09-25 10:06:21.0,IGLESIA CATOLICA,NaN,NaN,NaN,50450.0,14.0,15.0,"AVENIDA FABELA ISIDRO NTE,# S/N,COL.CENTRO,CP....",1.0,528.0,NaN,NaN,NaN,NaN,2008.0,1.0,12.0,4697952.0,Pago Municipal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,left_only,0240100201000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151022,0240721201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TM049657,GARCIA VALVERDE PATRICIA,XAXX010101000,"CAMINO SANTA CRUZ, S/N, LA PALMA",NaN,579.00,NaN,115943.0,DE 01-2022 AL 12-2022,2022,2022-12-28,240.0,40.0,NaN,NaN,3A6E68CF-C68C-4295-AEE7-253BA383E5F7,NaN,0240721200,0240721201,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-99.87280 19.81319, -99.87388 19.813...",024,07,212,right_only,0240721201000000
151023,0240721202,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TM049694,GARCIA VALVERDE ALICIA,XAXX010101000,"CERRADA SIN NOMBRE, S/N, LA PALMA",NaN,565.00,NaN,112924.0,DE 01-2022 AL 12-2022,2022,2022-12-29,238.0,40.0,NaN,NaN,ED9A0E0A-267D-44DD-9438-E4717F8721C1,NaN,0240721200,0240721202,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-99.87280 19.81319, -99.87388 19.813...",024,07,212,right_only,0240721202000000
4967,0240156004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TM000700,MONROY VELASCO ENRIQUE

In [86]:
len(base_final_CURT_Predial.CLAVECATASTRAL.unique())

31001

In [111]:
#exportar base 
base_final_CURT_Predial.to_csv(r"I:\Mi unidad\Cruces_predial_otros\Atlacomulco\Bases_salida_predial/Atlacomulco_crucexcurtxpredial.csv",encoding='utf-8-sig',index=False)


In [92]:
base_final_CURT_Predial = base_final_CURT_Predial.set_geometry('geometry')
base_final_CURT_Predial['geometry']=base_final_CURT_Predial.centroid

In [93]:
base_final_CURT_Predial['LATITUD']= base_final_CURT_Predial['geometry'].y
base_final_CURT_Predial['LONGITUD']= base_final_CURT_Predial['geometry'].x
base_final_CURT_Predial

,CLAVE_CATASTRAL,CLAVEMANZANA,CLAVEZONA,DIRECCION,VALORTERRENO,VALORTERRENOCOMUN,VALORCONSTPROP,VALORCONSTCOMUN,SUPERFICIE,SUPERFICIETERRCOMUN,SUPERFICIECONST,SUPERFICIECONSTCOMUN,CLAVEANTERIOR,FECHAALTA,NOMBRE,APATERNO,AMATERNO,RFC,CLAVECP,CLAVEMUNICIPIO,CLAVEENTIDAD,CALLE,CLAVESTATUS,CLAVEASENTAMIENTO,EMAIL,CURP,ASENTAMIENTO_NR,CLAVECP_NR,EJERCICIO,CLAVEPERIODOINI,CLAVEPERIODOFIN,VALORCATASTRAL,STATUS,fol_rec,nombre,rfc,ubicacion,domicilio_fiscal,sup_terreno,sup_construc,valor_catastral,periodo,periodo_anual,fec_pago,importe,recargos,multas,descuento,fol_fisc_uuid,facturado,CLAVE_MANZ,CLAVE_PREDIO,cve_ent,cve_mun,cve_loc,curt,notas,geometry,mun,zona,manz,ID_predial,CLAVECATASTRAL,LATITUD,LONGITUD
0,0240000000,NaN,NaN,PRUEBA 1 1,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,2013-09-25 12:59:34.0,PRUEBA 1,NaN,NaN,NaN,51260.0,7.0,15.0,PRUEBA 1,1.0,1054.0,NaN,NaN,NaN,NaN,2018.0,1.0,12.0,245321.0,Adeudo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,left_only,0240000000PRUEBA,NaN,NaN
1,0240000001,NaN,NaN,PRUEBA 2 2,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,NaN,2013-09-25 13:33:11.0,PRUEBA 2,NaN,NaN,NaN,56766.0,17.0,15.0,PRUEBA 2 2,1.0,9282.0,NaN,NaN,NaN,NaN,2012.0,1.0,12.0,22222.0,Adeudo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,left_only,0240000001PRUEBA,NaN,NaN
2,0240000004,NaN,NaN,PRUEBA 111 111,300000,NaN,145000,NaN,100.0,NaN,200.0,NaN,NaN,2013-10-10 10:47:42.0,PRUEBA 1111,PRUEBA,PRUEBA,NaN,52542.0,6.0,15.0,PRUEBA 111 111,1.0,6881.0,NaN,NaN,NaN,NaN,2011.0,1.0,12.0,400000.0,Adeudo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,left_only,0240000004PRUEBA,NaN,NaN
3,0240100101,1.0,1.0,ISIDRO FABELA SUR S/N,5245887,NaN,NaN,NaN,3813.0,NaN,NaN,NaN,NaN,2014-11-03 11:27:32.0,H. AYUNTAMIENTO,DE ATLACOMULCO,(PLAZA MIGUEL HIDALGO),NaN,50450.0,14.0,15.0,AV. ISIDRO FABELA SUR S/N,1.0,528.0,NaN,MAT,NaN,NaN,2015.0,1.0,12.0,4348116.0,Adeudo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,left_only,0240100101000000,NaN,NaN
4,0240100201,2.0,1.0,"AVENIDA FABELA ISIDRO NTE,# S/N,COL.CENT S/N S/N",4092285,NaN,3341520,NaN,5493.0,NaN,1560.0,NaN,0240100201000000,2013-09-25 10:06:21.0,IGLESIA CATOLICA,NaN,NaN,NaN,50450.0,14.0,15.0,"AVENIDA FABELA ISIDRO NTE,# S/N,COL.CENTRO,CP....",1.0,528.0,NaN,NaN,NaN,NaN,2008.0,1.0,12.0,4697952.0,Pago Municipal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,left_only,0240100201000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151022,0240721201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TM049657,GARCIA VALVERDE PATRICIA,XAXX010101000,"CAMINO SANTA CRUZ, S/N, LA PALMA",NaN,579.00,NaN,115943.0,DE 01-2022 AL 12-2022,2022,2022-12-28,240.0,40.0,NaN,NaN,3A6E68CF-C68C-4295-AEE7-253BA383E5F7,NaN,0240721200,0240721201,NaN,NaN,NaN,NaN,NaN,POINT (-99.87332 19.81364),024,07,212,right_only,0240721201000000,19.813642,-99.87332
151023,0240721202,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TM049694,GARCIA VALVERDE ALICIA,XAXX010101000,"CERRADA SIN NOMBRE, S/N, LA PALMA",NaN,565.00,NaN,112924.0,DE 01-2022 AL 12-2022,2022,2022-12-29,238.0,40.0,NaN,NaN,ED9A0E0A-267D-44DD-9438-E4717F8721C1,NaN,0240721200,0240721202,NaN,NaN,NaN,NaN,NaN,POINT (-99.87332 19.81364),024,07,212,right_only,0240721202000000,19.813642,-99.87332
4967,0240156004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [94]:
#Revisar si la base tiene LAT y LON
base_final_CURT_Predial[base_final_CURT_Predial.LONGITUD.isna()]


,CLAVE_CATASTRAL,CLAVEMANZANA,CLAVEZONA,DIRECCION,VALORTERRENO,VALORTERRENOCOMUN,VALORCONSTPROP,VALORCONSTCOMUN,SUPERFICIE,SUPERFICIETERRCOMUN,SUPERFICIECONST,SUPERFICIECONSTCOMUN,CLAVEANTERIOR,FECHAALTA,NOMBRE,APATERNO,AMATERNO,RFC,CLAVECP,CLAVEMUNICIPIO,CLAVEENTIDAD,CALLE,CLAVESTATUS,CLAVEASENTAMIENTO,EMAIL,CURP,ASENTAMIENTO_NR,CLAVECP_NR,EJERCICIO,CLAVEPERIODOINI,CLAVEPERIODOFIN,VALORCATASTRAL,STATUS,fol_rec,nombre,rfc,ubicacion,domicilio_fiscal,sup_terreno,sup_construc,valor_catastral,periodo,periodo_anual,fec_pago,importe,recargos,multas,descuento,fol_fisc_uuid,facturado,CLAVE_MANZ,CLAVE_PREDIO,cve_ent,cve_mun,cve_loc,curt,notas,geometry,mun,zona,manz,ID_predial,CLAVECATASTRAL,LATITUD,LONGITUD
0,0240000000,NaN,NaN,PRUEBA 1 1,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,2013-09-25 12:59:34.0,PRUEBA 1,NaN,NaN,NaN,51260.0,7.0,15.0,PRUEBA 1,1.0,1054.0,NaN,NaN,NaN,NaN,2018.0,1.0,12.0,245321.0,Adeudo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,left_only,0240000000PRUEBA,NaN,NaN
1,0240000001,NaN,NaN,PRUEBA 2 2,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,NaN,2013-09-25 13:33:11.0,PRUEBA 2,NaN,NaN,NaN,56766.0,17.0,15.0,PRUEBA 2 2,1.0,9282.0,NaN,NaN,NaN,NaN,2012.0,1.0,12.0,22222.0,Adeudo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,left_only,0240000001PRUEBA,NaN,NaN
2,0240000004,NaN,NaN,PRUEBA 111 111,300000,NaN,145000,NaN,100.0,NaN,200.0,NaN,NaN,2013-10-10 10:47:42.0,PRUEBA 1111,PRUEBA,PRUEBA,NaN,52542.0,6.0,15.0,PRUEBA 111 111,1.0,6881.0,NaN,NaN,NaN,NaN,2011.0,1.0,12.0,400000.0,Adeudo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,left_only,0240000004PRUEBA,NaN,NaN
3,0240100101,1.0,1.0,ISIDRO FABELA SUR S/N,5245887,NaN,NaN,NaN,3813.0,NaN,NaN,NaN,NaN,2014-11-03 11:27:32.0,H. AYUNTAMIENTO,DE ATLACOMULCO,(PLAZA MIGUEL HIDALGO),NaN,50450.0,14.0,15.0,AV. ISIDRO FABELA SUR S/N,1.0,528.0,NaN,MAT,NaN,NaN,2015.0,1.0,12.0,4348116.0,Adeudo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,left_only,0240100101000000,NaN,NaN
4,0240100201,2.0,1.0,"AVENIDA FABELA ISIDRO NTE,# S/N,COL.CENT S/N S/N",4092285,NaN,3341520,NaN,5493.0,NaN,1560.0,NaN,0240100201000000,2013-09-25 10:06:21.0,IGLESIA CATOLICA,NaN,NaN,NaN,50450.0,14.0,15.0,"AVENIDA FABELA ISIDRO NTE,# S/N,COL.CENTRO,CP....",1.0,528.0,NaN,NaN,NaN,NaN,2008.0,1.0,12.0,4697952.0,Pago Municipal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,left_only,0240100201000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149774,0241530819,308.0,15.0,A SAN IGNACIO Y LA JOYA S/N,27306,NaN,NaN,NaN,246.0,NaN,NaN,NaN,0241530801000000,2014-09-24 09:50:06.0,MARÍA DE JESÚS,ORTEGA,MARTÍNEZ,OEMJ550520,50475.0,14.0,15.0,A SAN IGNACIO S/N,1.0,555.0,NaN,OEMJ550520MMCRR504,NaN,NaN,2015.0,1.0,12.0,23370.0,Pago Municipal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,left_only,0241530819000000,NaN,NaN
149775,0241530820,308.0,15.0,A SAN IGNACIO Y LA JOYA S/N S/N,83250,NaN,NaN,NaN,750.0,NaN,NaN,NaN,0241530801000000,2014-12-04 13:52:59.0,FABIAN,NAVARRO,FLORES,NAFF760728ES5,50475.0,14.0,15.0,A SAN IGNACIO Y LA JOYA S/N S/N,1.0,555.0,NaN,NAFF760718HMCVLB07,NaN,NaN,2015.0,1.0,12.0,71250.0,Adeudo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,left_only,0241530820000000,NaN,NaN
4967,0240156004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TM000700,MONROY VELASCO ENRIQUE,XAXX010101000,"AV. DE

Agregar LAT LON a los registros que no venían en la base de CURT

In [95]:
base_final_CURT_Predial.loc[base_final_CURT_Predial['LATITUD'].isna(), 'LATITUD'] = 18
base_final_CURT_Predial.loc[base_final_CURT_Predial['LONGITUD'].isna(), 'LONGITUD'] = -99

In [96]:
base_final_CURT_Predial.drop("geometry",axis=1,inplace=True)

In [97]:
base_final_CURT_Predial = prep.transform_df_to_gpd(base_final_CURT_Predial, 'LONGITUD','LATITUD', crs= 'EPSG:4326')
base_final_CURT_Predial

,CLAVE_CATASTRAL,CLAVEMANZANA,CLAVEZONA,DIRECCION,VALORTERRENO,VALORTERRENOCOMUN,VALORCONSTPROP,VALORCONSTCOMUN,SUPERFICIE,SUPERFICIETERRCOMUN,SUPERFICIECONST,SUPERFICIECONSTCOMUN,CLAVEANTERIOR,FECHAALTA,NOMBRE,APATERNO,AMATERNO,RFC,CLAVECP,CLAVEMUNICIPIO,CLAVEENTIDAD,CALLE,CLAVESTATUS,CLAVEASENTAMIENTO,EMAIL,CURP,ASENTAMIENTO_NR,CLAVECP_NR,EJERCICIO,CLAVEPERIODOINI,CLAVEPERIODOFIN,VALORCATASTRAL,STATUS,fol_rec,nombre,rfc,ubicacion,domicilio_fiscal,sup_terreno,sup_construc,valor_catastral,periodo,periodo_anual,fec_pago,importe,recargos,multas,descuento,fol_fisc_uuid,facturado,CLAVE_MANZ,CLAVE_PREDIO,cve_ent,cve_mun,cve_loc,curt,notas,mun,zona,manz,ID_predial,CLAVECATASTRAL,LATITUD,LONGITUD,geometry
0,0240000000,NaN,NaN,PRUEBA 1 1,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,2013-09-25 12:59:34.0,PRUEBA 1,NaN,NaN,NaN,51260.0,7.0,15.0,PRUEBA 1,1.0,1054.0,NaN,NaN,NaN,NaN,2018.0,1.0,12.0,245321.0,Adeudo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,0240000000PRUEBA,18.000000,-99.00000,POINT (-99.00000 18.00000)
1,0240000001,NaN,NaN,PRUEBA 2 2,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,NaN,2013-09-25 13:33:11.0,PRUEBA 2,NaN,NaN,NaN,56766.0,17.0,15.0,PRUEBA 2 2,1.0,9282.0,NaN,NaN,NaN,NaN,2012.0,1.0,12.0,22222.0,Adeudo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,0240000001PRUEBA,18.000000,-99.00000,POINT (-99.00000 18.00000)
2,0240000004,NaN,NaN,PRUEBA 111 111,300000,NaN,145000,NaN,100.0,NaN,200.0,NaN,NaN,2013-10-10 10:47:42.0,PRUEBA 1111,PRUEBA,PRUEBA,NaN,52542.0,6.0,15.0,PRUEBA 111 111,1.0,6881.0,NaN,NaN,NaN,NaN,2011.0,1.0,12.0,400000.0,Adeudo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,0240000004PRUEBA,18.000000,-99.00000,POINT (-99.00000 18.00000)
3,0240100101,1.0,1.0,ISIDRO FABELA SUR S/N,5245887,NaN,NaN,NaN,3813.0,NaN,NaN,NaN,NaN,2014-11-03 11:27:32.0,H. AYUNTAMIENTO,DE ATLACOMULCO,(PLAZA MIGUEL HIDALGO),NaN,50450.0,14.0,15.0,AV. ISIDRO FABELA SUR S/N,1.0,528.0,NaN,MAT,NaN,NaN,2015.0,1.0,12.0,4348116.0,Adeudo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,0240100101000000,18.000000,-99.00000,POINT (-99.00000 18.00000)
4,0240100201,2.0,1.0,"AVENIDA FABELA ISIDRO NTE,# S/N,COL.CENT S/N S/N",4092285,NaN,3341520,NaN,5493.0,NaN,1560.0,NaN,0240100201000000,2013-09-25 10:06:21.0,IGLESIA CATOLICA,NaN,NaN,NaN,50450.0,14.0,15.0,"AVENIDA FABELA ISIDRO NTE,# S/N,COL.CENTRO,CP....",1.0,528.0,NaN,NaN,NaN,NaN,2008.0,1.0,12.0,4697952.0,Pago Municipal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,0240100201000000,18.000000,-99.00000,POINT (-99.00000 18.00000)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151022,0240721201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TM049657,GARCIA VALVERDE PATRICIA,XAXX010101000,"CAMINO SANTA CRUZ, S/N, LA PALMA",NaN,579.00,NaN,115943.0,DE 01-2022 AL 12-2022,2022,2022-12-28,240.0,40.0,NaN,NaN,3A6E68CF-C68C-4295-AEE7-253BA383E5F7,NaN,0240721200,0240721201,NaN,NaN,NaN,NaN,NaN,024,07,212,right_only,0240721201000000,19.813642,-99.87332,POINT (-99.87332 19.81364)
151023,0240721202,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TM049694,GARCIA VALVERDE ALICIA,XAXX010101000,"CERRADA SIN NOMBRE, S/N, LA PALMA",NaN,565.00,NaN,112924.0,DE 01-2022 AL 12-2022,2022,2022-12-29,238.0,40.0,NaN,NaN,ED9A0E0A-267D-44DD-9438-E4717F8721C1,NaN,0240721200,0240721202,NaN,NaN,NaN,NaN,NaN,024,07,212,right_only,02407212020000

In [98]:
base_final_CURT_Predial[base_final_CURT_Predial.geometry.isna()]

,CLAVE_CATASTRAL,CLAVEMANZANA,CLAVEZONA,DIRECCION,VALORTERRENO,VALORTERRENOCOMUN,VALORCONSTPROP,VALORCONSTCOMUN,SUPERFICIE,SUPERFICIETERRCOMUN,SUPERFICIECONST,SUPERFICIECONSTCOMUN,CLAVEANTERIOR,FECHAALTA,NOMBRE,APATERNO,AMATERNO,RFC,CLAVECP,CLAVEMUNICIPIO,CLAVEENTIDAD,CALLE,CLAVESTATUS,CLAVEASENTAMIENTO,EMAIL,CURP,ASENTAMIENTO_NR,CLAVECP_NR,EJERCICIO,CLAVEPERIODOINI,CLAVEPERIODOFIN,VALORCATASTRAL,STATUS,fol_rec,nombre,rfc,ubicacion,domicilio_fiscal,sup_terreno,sup_construc,valor_catastral,periodo,periodo_anual,fec_pago,importe,recargos,multas,descuento,fol_fisc_uuid,facturado,CLAVE_MANZ,CLAVE_PREDIO,cve_ent,cve_mun,cve_loc,curt,notas,mun,zona,manz,ID_predial,CLAVECATASTRAL,LATITUD,LONGITUD,geometry


In [99]:
base_temp=base_final_CURT_Predial.groupby('CLAVE_CATASTRAL').count()[['CLAVECATASTRAL','curt']].reset_index()
base_temp

,CLAVE_CATASTRAL,CLAVECATASTRAL,curt
0,0240000000,1,0
1,0240000001,1,0
2,0240000004,1,0
3,0240100101,1,0
4,0240100201,1,0
...,...,...,...
22045,0241532018,1,0
22046,0241532021,1,0
22047,0241532022,1,0
22048,0241532024,1,0


In [100]:
base_temp=base_temp.rename(columns={'CLAVECATASTRAL':'ESTIMADO'})
base_temp

,CLAVE_CATASTRAL,ESTIMADO,curt
0,0240000000,1,0
1,0240000001,1,0
2,0240000004,1,0
3,0240100101,1,0
4,0240100201,1,0
...,...,...,...
22045,0241532018,1,0
22046,0241532021,1,0
22047,0241532022,1,0
22048,0241532024,1,0


In [101]:
crucexcurt=base_final_CURT_Predial.merge(base_temp[['ESTIMADO','curt','CLAVE_CATASTRAL']],on='CLAVE_CATASTRAL')
crucexcurt

,CLAVE_CATASTRAL,CLAVEMANZANA,CLAVEZONA,DIRECCION,VALORTERRENO,VALORTERRENOCOMUN,VALORCONSTPROP,VALORCONSTCOMUN,SUPERFICIE,SUPERFICIETERRCOMUN,SUPERFICIECONST,SUPERFICIECONSTCOMUN,CLAVEANTERIOR,FECHAALTA,NOMBRE,APATERNO,AMATERNO,RFC,CLAVECP,CLAVEMUNICIPIO,CLAVEENTIDAD,CALLE,CLAVESTATUS,CLAVEASENTAMIENTO,EMAIL,CURP,ASENTAMIENTO_NR,CLAVECP_NR,EJERCICIO,CLAVEPERIODOINI,CLAVEPERIODOFIN,VALORCATASTRAL,STATUS,fol_rec,nombre,rfc,ubicacion,domicilio_fiscal,sup_terreno,sup_construc,valor_catastral,periodo,periodo_anual,fec_pago,importe,recargos,multas,descuento,fol_fisc_uuid,facturado,CLAVE_MANZ,CLAVE_PREDIO,cve_ent,cve_mun,cve_loc,curt_x,notas,mun,zona,manz,ID_predial,CLAVECATASTRAL,LATITUD,LONGITUD,geometry,ESTIMADO,curt_y
0,0240000000,NaN,NaN,PRUEBA 1 1,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,2013-09-25 12:59:34.0,PRUEBA 1,NaN,NaN,NaN,51260.0,7.0,15.0,PRUEBA 1,1.0,1054.0,NaN,NaN,NaN,NaN,2018.0,1.0,12.0,245321.0,Adeudo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,0240000000PRUEBA,18.000000,-99.000000,POINT (-99.00000 18.00000),1,0
1,0240000001,NaN,NaN,PRUEBA 2 2,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,NaN,2013-09-25 13:33:11.0,PRUEBA 2,NaN,NaN,NaN,56766.0,17.0,15.0,PRUEBA 2 2,1.0,9282.0,NaN,NaN,NaN,NaN,2012.0,1.0,12.0,22222.0,Adeudo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,0240000001PRUEBA,18.000000,-99.000000,POINT (-99.00000 18.00000),1,0
2,0240000004,NaN,NaN,PRUEBA 111 111,300000,NaN,145000,NaN,100.0,NaN,200.0,NaN,NaN,2013-10-10 10:47:42.0,PRUEBA 1111,PRUEBA,PRUEBA,NaN,52542.0,6.0,15.0,PRUEBA 111 111,1.0,6881.0,NaN,NaN,NaN,NaN,2011.0,1.0,12.0,400000.0,Adeudo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,0240000004PRUEBA,18.000000,-99.000000,POINT (-99.00000 18.00000),1,0
3,0240100101,1.0,1.0,ISIDRO FABELA SUR S/N,5245887,NaN,NaN,NaN,3813.0,NaN,NaN,NaN,NaN,2014-11-03 11:27:32.0,H. AYUNTAMIENTO,DE ATLACOMULCO,(PLAZA MIGUEL HIDALGO),NaN,50450.0,14.0,15.0,AV. ISIDRO FABELA SUR S/N,1.0,528.0,NaN,MAT,NaN,NaN,2015.0,1.0,12.0,4348116.0,Adeudo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,0240100101000000,18.000000,-99.000000,POINT (-99.00000 18.00000),1,0
4,0240100201,2.0,1.0,"AVENIDA FABELA ISIDRO NTE,# S/N,COL.CENT S/N S/N",4092285,NaN,3341520,NaN,5493.0,NaN,1560.0,NaN,0240100201000000,2013-09-25 10:06:21.0,IGLESIA CATOLICA,NaN,NaN,NaN,50450.0,14.0,15.0,"AVENIDA FABELA ISIDRO NTE,# S/N,COL.CENTRO,CP....",1.0,528.0,NaN,NaN,NaN,NaN,2008.0,1.0,12.0,4697952.0,Pago Municipal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,0240100201000000,18.000000,-99.000000,POINT (-99.00000 18.00000),1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151022,0240326401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TM051561,MARIA MENDOZA VDA. DE SERRANO Y JUAN SERRANO M...,XAXX010101000,CALLE SIN NOMBRE SAN LORENZO TLACOTEPEC,NaN,4934.0,NaN,1393273.0,DE 01-2017 AL 12-2017,2017,2022-11-23,3294.0,1644.0,NaN,1842.0,058DCB3A-CE41-451B-B86A-7D38CA6D638F,NaN,0240326400,0240326401,NaN,NaN,NaN,NaN,NaN,024,03,264,right_only,0240326401000000,19.812918,-99.917230,POINT (-99.91723 19.81292),1,0
151023,0240405203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TM051701,GUTIERREZ MIRA EPITACIA CONCEPCION,XAXX010101000,"CALLE 5 DE MAYO, S/N, CUENDO",NaN,657.0,NaN,12155.0,DE 01-2022 AL 12-2022,2022,2022-11-29,200.0,30.0,NaN,18.0,FC602273-F2CE-4C21-B

In [103]:
# crucexcurt.drop("curt_y",axis=1,inplace=True)
crucexcurt.rename(columns={'curt_y':'CURT', "curt_x":"curt"},inplace=True)
crucexcurt

,CLAVE_CATASTRAL,CLAVEMANZANA,CLAVEZONA,DIRECCION,VALORTERRENO,VALORTERRENOCOMUN,VALORCONSTPROP,VALORCONSTCOMUN,SUPERFICIE,SUPERFICIETERRCOMUN,SUPERFICIECONST,SUPERFICIECONSTCOMUN,CLAVEANTERIOR,FECHAALTA,NOMBRE,APATERNO,AMATERNO,RFC,CLAVECP,CLAVEMUNICIPIO,CLAVEENTIDAD,CALLE,CLAVESTATUS,CLAVEASENTAMIENTO,EMAIL,CURP,ASENTAMIENTO_NR,CLAVECP_NR,EJERCICIO,CLAVEPERIODOINI,CLAVEPERIODOFIN,VALORCATASTRAL,STATUS,fol_rec,nombre,rfc,ubicacion,domicilio_fiscal,sup_terreno,sup_construc,valor_catastral,periodo,periodo_anual,fec_pago,importe,recargos,multas,descuento,fol_fisc_uuid,facturado,CLAVE_MANZ,CLAVE_PREDIO,cve_ent,cve_mun,cve_loc,curt,notas,mun,zona,manz,ID_predial,CLAVECATASTRAL,LATITUD,LONGITUD,geometry,ESTIMADO,CURT
0,0240000000,NaN,NaN,PRUEBA 1 1,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,2013-09-25 12:59:34.0,PRUEBA 1,NaN,NaN,NaN,51260.0,7.0,15.0,PRUEBA 1,1.0,1054.0,NaN,NaN,NaN,NaN,2018.0,1.0,12.0,245321.0,Adeudo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,0240000000PRUEBA,18.000000,-99.000000,POINT (-99.00000 18.00000),1,0
1,0240000001,NaN,NaN,PRUEBA 2 2,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,NaN,2013-09-25 13:33:11.0,PRUEBA 2,NaN,NaN,NaN,56766.0,17.0,15.0,PRUEBA 2 2,1.0,9282.0,NaN,NaN,NaN,NaN,2012.0,1.0,12.0,22222.0,Adeudo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,0240000001PRUEBA,18.000000,-99.000000,POINT (-99.00000 18.00000),1,0
2,0240000004,NaN,NaN,PRUEBA 111 111,300000,NaN,145000,NaN,100.0,NaN,200.0,NaN,NaN,2013-10-10 10:47:42.0,PRUEBA 1111,PRUEBA,PRUEBA,NaN,52542.0,6.0,15.0,PRUEBA 111 111,1.0,6881.0,NaN,NaN,NaN,NaN,2011.0,1.0,12.0,400000.0,Adeudo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,0240000004PRUEBA,18.000000,-99.000000,POINT (-99.00000 18.00000),1,0
3,0240100101,1.0,1.0,ISIDRO FABELA SUR S/N,5245887,NaN,NaN,NaN,3813.0,NaN,NaN,NaN,NaN,2014-11-03 11:27:32.0,H. AYUNTAMIENTO,DE ATLACOMULCO,(PLAZA MIGUEL HIDALGO),NaN,50450.0,14.0,15.0,AV. ISIDRO FABELA SUR S/N,1.0,528.0,NaN,MAT,NaN,NaN,2015.0,1.0,12.0,4348116.0,Adeudo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,0240100101000000,18.000000,-99.000000,POINT (-99.00000 18.00000),1,0
4,0240100201,2.0,1.0,"AVENIDA FABELA ISIDRO NTE,# S/N,COL.CENT S/N S/N",4092285,NaN,3341520,NaN,5493.0,NaN,1560.0,NaN,0240100201000000,2013-09-25 10:06:21.0,IGLESIA CATOLICA,NaN,NaN,NaN,50450.0,14.0,15.0,"AVENIDA FABELA ISIDRO NTE,# S/N,COL.CENTRO,CP....",1.0,528.0,NaN,NaN,NaN,NaN,2008.0,1.0,12.0,4697952.0,Pago Municipal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,0240100201000000,18.000000,-99.000000,POINT (-99.00000 18.00000),1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151022,0240326401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TM051561,MARIA MENDOZA VDA. DE SERRANO Y JUAN SERRANO M...,XAXX010101000,CALLE SIN NOMBRE SAN LORENZO TLACOTEPEC,NaN,4934.0,NaN,1393273.0,DE 01-2017 AL 12-2017,2017,2022-11-23,3294.0,1644.0,NaN,1842.0,058DCB3A-CE41-451B-B86A-7D38CA6D638F,NaN,0240326400,0240326401,NaN,NaN,NaN,NaN,NaN,024,03,264,right_only,0240326401000000,19.812918,-99.917230,POINT (-99.91723 19.81292),1,0
151023,0240405203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TM051701,GUTIERREZ MIRA EPITACIA CONCEPCION,XAXX010101000,"CALLE 5 DE MAYO, S/N, CUENDO",NaN,657.0,NaN,12155.0,DE 01-2022 AL 12-2022,2022,2022-11-29,200.0,30.0,NaN,18.0,FC602273-F2CE-4C21-B534-

In [104]:
len(crucexcurt.CLAVECATASTRAL.unique())

31001

In [105]:
no_predial=base_predial[~(base_predial.CLAVECATASTRAL.isin(crucexcurt['CLAVECATASTRAL']))]
len(no_predial.CLAVECATASTRAL.unique())

0

In [133]:
crucexcurt.to_csv(r'I:\Mi unidad\Cruces_predial_otros\Ixtapan/Ixtapan_Base_predial_PREGEO.csv',encoding="utf-8-sig", index=False)